# Scraping executions in 50 states since 1976

The notation for each dataframe is consistent: 
- the dataframe with all the links to scrape is called df_stateName
- the dataframe with all the scraped information is called df_all_stateName
- we merge these two dfs, and call them df_firstThreeLettersOfStateName
- once cleaned, df_firstThreeLettersOfStateName is exported as stateName_final.csv

The distilled information should at least contain the name, number of victims, date of offense, date of execution, race, gender, state, and final words. The dataframes contain a bunch of garbage columns that are not needed, and that will be taken care of in another notebook: in the end we'll only want 8 columns for each state df, and we'll concatenate all of them to have an overall df of all executions in the US since 1976. 

WARNING: when the information we are interested in was missing (noted 'NOT FOUND' in the scraped df), it was retrieved by hand - by reading the multiple press releases, and articles included on the convict's webpage on murderopedia, when it came to dates and last words. For race and gender, deathpenalty.org execution database was used. 

NOTE: final_words are usually spoke, except when precised otherwise. A few written statements are in there, but they are formatted as letters, so the difference between written / spoken statements should be rather straightforward to readers. 

We'll be able to use this overall df in d3 - and we'll also create multiple smaller dfs for each year we have data for, and each gender, for the purposes of a search bar (I know there's a way to optimize this but for now oh well.) 


In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import re

In [9]:
def scrape_each_execution(row):
    driver.get(row['link_to_profile'])
    try:
        table= driver.find_element_by_id('AutoNumber3')
    except:
        pass
    try:
        table = driver.find_elements_by_tag_name('table')[13]
    except:
        pass
    
    row = table.find_elements_by_tag_name('tr')[0].text

    convict = {}
    try:
        offense = re.findall(r"Characteristics: (.*)", row)[0]
    except:
        offense = 'NOT FOUND'
    try:
        date_offense = re.findall(r"Date of murders?: (.*)", row)[0]
    except:
        date_offense = 'NOT FOUND'
    try:
        date_arrest = re.findall(r"Date of arrest: (.*)", row)[0]
    except:
        date_arrest = 'NOT FOUND'
    try:
        date_of_birth = re.findall(r"Date of birth: (.*)", row)[0]
    except:
        date_of_birth = 'NOT FOUND'
    try:
        victim_num = re.findall(r"Number of victims: (.*)", row)[0]
    except:
        victim_num = 'NOT FOUND'
    try:
        method = re.findall(r"Status: (.*)", row)[0]
    except:
        method = 'NOT FOUND'
    try:
        race = re.findall(r"DOC#: \d*[\n\s](.*)\n", row)[0]
    except:
        race = 'NOT FOUND'
    try:
        final_words = re.findall(r"Final Words:\n\n(.*)\n", row)[0]
    except:
        final_words = 'NOT FOUND'
    try:
        mitigating_circumstances = re.findall(r"Mitigating Circumstances: (.*)", row)[0]
    except:
        mitigating_circumstances = 'NOT FOUND'
    return pd.Series({
         'offense': offense,
        'date_offense': date_offense,
        'date_arrest':date_arrest,
     'date_of_birth':date_of_birth,
     'victim_num':victim_num,
     'method':method,
    'race': race,
    'final_words': final_words,
    'mitigating_circumstances' : mitigating_circumstances
     })

# Alabama

In [10]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/alabama.htm')

In [ ]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Alabama"
    rows_df.append(row_df)
#print(rows_df)
df_alabama = pd.DataFrame(rows_df)
df_alabama

In [ ]:
df_all_alabama = df_alabama.apply(scrape_each_execution, axis=1)
df_all_alabama

In [ ]:
df_all_alabama['url'] = df_alabama.link_to_profile 
df_all_alabama = df_all_alabama.merge(df_alabama, right_on='link_to_profile', left_on = 'url')
df_all_alabama

In [15]:
df_all_alabama.to_csv('alabama-2.csv', index=False)
#df_all_alabama = pd.read_csv('alabama-2.csv', na_values=['NOT FOUND'])
#df_all_alabama = df_all_alabama.fillna('Unknown')
#df_all_alabama

In [ ]:
df_all_alabama['gender'] = df_all_alabama['race'].str.extract(r".*\s(.*)")
df_all_alabama['race_new'] = df_all_alabama['race'].str.extract(r"(.*)\s")
df_all_alabama['year'] = df_all_alabama['date_execution_y'].str.extract(r"(\d\d\d\d)")
df_all_alabama

In [ ]:
df_all_alabama.to_csv('final_alabama.csv', index=False)

In [ ]:
df_ala = pd.read_csv('alabama_2_bis.csv', encoding = 'latin1', na_values=['NOT FOUND'])
df_ala['date_exe'] = df_all_alabama['date_execution']
df_ala['gender'] = df_ala['race'].str.extract(r".*\s(.*)")
df_ala['race_new'] = df_ala['race'].str.extract(r"(.*)\s")
df_ala['year'] = df_ala['date_exe'].str.extract(r"(\d\d\d\d)")
df_ala = df_ala.fillna('Unknown')
df_ala['date_offense'] = df_all_alabama['date_offense']

df_ala

In [27]:
df_ala.to_csv('final_alabama.csv', index=False)

# Texas

In [32]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/texas.htm')

In [ ]:
table = driver.find_element_by_id('table14')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[3].text
    row_df['race'] = cells[4].text
    row_df['method'] = cells[5].text
    row_df['county'] = cells[6].text
    row_df['State'] = "Texas"
    rows_df.append(row_df)
#print(rows_df)
df_texas = pd.DataFrame(rows_df)
df_texas

In [51]:
def scrape_each_execution_texas(row):
    driver.get(row['link_to_profile'])
    try:
        table= driver.find_element_by_id('AutoNumber3')
    except:
        pass
    try:
        table = driver.find_elements_by_tag_name('table')[13]
    except:
        pass
    
    row = table.find_elements_by_tag_name('tr')[0].text

    convict = {}
    try:
        offense = re.findall(r"Characteristics: (.*)", row)[0]
    except:
        offense = 'NOT FOUND'
    try:
        date_offense = re.findall(r"Date of murders?: (.*)", row)[0]
    except:
        date_offense = 'NOT FOUND'
    try:
        date_arrest = re.findall(r"Date of arrest: (.*)", row)[0]
    except:
        date_arrest = 'NOT FOUND'
    try:
        date_of_birth = re.findall(r"Date of birth: (.*)", row)[0]
    except:
        date_of_birth = 'NOT FOUND'
    try:
        victim_num = re.findall(r"Number of victims: (.*)", row)[0]
    except:
        victim_num = 'NOT FOUND'
    try:
        method = re.findall(r"Status: (.*)", row)[0]
    except:
        method = 'NOT FOUND'
    try:
        race = re.findall(r"DOC#: \d*[\n\s](.*)\n", row)[0]
    except:
        race = 'NOT FOUND'
    try:
        final_words = re.findall(r"Words:\n\n(.*)\n", row)[0]
    except:
        final_words = 'NOT FOUND'
    try:
        mitigating_circumstances = re.findall(r"Mitigating Circumstances: (.*)", row)[0]
    except:
        mitigating_circumstances = 'NOT FOUND'
    return pd.Series({
         'offense': offense,
        'date_offense': date_offense,
        'date_arrest':date_arrest,
     'date_of_birth':date_of_birth,
     'victim_num':victim_num,
     'method':method,
    'race': race,
    'final_words': final_words,
    'mitigating_circumstances' : mitigating_circumstances
     })

In [55]:
driver = webdriver.Chrome()

df_all_texas = df_texas.apply(scrape_each_execution_texas, axis=1)
df_all_texas

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,NOT FOUND,"October 6, 1975","July 26, 2004","""I'm all right. Tell them I finished strong. G...","Sentenced to death on June 22, 2006. Executed ...",NOT FOUND,Child abuse - Torture,NOT FOUND,1
1,Same day,"September 8, 1969","May 3, 1993","""I love you all. I'm going home, going to be w...","Sentenced to death on December 15, 1993. Execu...",NOT FOUND,Shooting rampage,NOT FOUND,2
2,Same day,"April 14, 1975","January 22, 2001","""I would like to remind my children once again...","Sentenced to death on May 21, 2002. Executed b...",NOT FOUND,Parricide - Drugs - Robbery,NOT FOUND,3
3,NOT FOUND,"October 5, 1969",1997 / 1999,"""I'm happy,"" Hernandez said in Spanish. ""I am ...","Sentenced to death on February 11, 2000. Execu...",NOT FOUND,Rape,NOT FOUND,2
4,"January 2, 2000","June 28, 1964",1980 - 1999,Declined.,"Sentenced to death in Texas on November 8, 200...",NOT FOUND,Rape,NOT FOUND,1 - 13 +
5,NOT FOUND,"October 16, 1984","January 16, 2003",NOT FOUND,"Sentenced to death on May 28, 2004. Executed b...",NOT FOUND,Robbery,NOT FOUND,1
6,Same day,"August 25, 1980","November 29, 1998",NOT FOUND,"Sentenced to death on February 4, 2000. Execut...",NOT FOUND,Rapper - Robbery,NOT FOUND,1
7,NOT FOUND,"May 15, 1954","August 25, 1998",NOT FOUND,"Sentenced to death on September 1, 1999. Execu...",NOT FOUND,Kidnapping - Torture - To collect insurance money,NOT FOUND,1
8,Same day,"July 22, 1967","January 31, 1994",NOT FOUND,"Sentenced to death on November 18, 1994. Execu...",NOT FOUND,Escape attempt,NOT FOUND,1
9,Same day,"March 28, 1970","September 24, 2007",NOT FOUND,"Sentenced to death on February 17, 2009. Execu...",NOT FOUND,Prison break,NOT FOUND,1


In [56]:
df_all_texas['url'] = df_texas.link_to_profile 
df_all_texas = df_all_texas.merge(df_texas, right_on='link_to_profile', left_on = 'url')
df_all_texas

,date_arrest,date_of_birth,date_offense,final_words,method_x,mitigating_circumstances,offense,race_x,victim_num,url,State,county,date_execution,link_to_profile,method_y,name,race_y
0,NOT FOUND,"October 6, 1975","July 26, 2004","""I'm all right. Tell them I finished strong. G...","Sentenced to death on June 22, 2006. Executed ...",NOT FOUND,Child abuse - Torture,NOT FOUND,1,http://murderpedia.org/female.C/c/coleman-lisa...,Texas,Tarrant,09/17/2014,http://murderpedia.org/female.C/c/coleman-lisa...,Lethal Injection,Lisa Ann Coleman,Black
1,Same day,"September 8, 1969","May 3, 1993","""I love you all. I'm going home, going to be w...","Sentenced to death on December 15, 1993. Execu...",NOT FOUND,Shooting rampage,NOT FOUND,2,http://murderpedia.org/male.T/t/trottie-willie...,Texas,Harris,09/10/2014,http://murderpedia.org/male.T/t/trottie-willie...,Lethal Injection,Willie Tyrone Trottie,Black
2,Same day,"April 14, 1975","January 22, 2001","""I would like to remind my children once again...","Sentenced to death on May 21, 2002. Executed b...",NOT FOUND,Parricide - Drugs - Robbery,NOT FOUND,3,http://murderpedia.org/male.V/v/villegas-jose-...,Texas,Nueces,04/16/2014,http://murderpedia.org/male.V/v/villegas-jose-...,Lethal Injection,Jose Luis Villegas Jr.,Hispanic
3,NOT FOUND,"October 5, 1969",1997 / 1999,"""I'm happy,"" Hernandez said in Spanish. ""I am ...","Sentenced to death on February 11, 2000. Execu...",NOT FOUND,Rape,NOT FOUND,2,http://murderpedia.org/male.H/h/hernandez-rami...,Texas,Bandera,04/09/2014,http://murderpedia.org/male.H/h/hernandez-rami...,Lethal Injection,Ramiro Hernandez,Hispanic
4,"January 2, 2000","June 28, 1964",1980 - 1999,Declined.,"Sentenced to death in Texas on November 8, 200...",NOT FOUND,Rape,NOT FOUND,1 - 13 +,http://murderpedia.org/male.S/s/sells-tommy-ly...,Texas,Val Verde,04/03/2014,http://murderpedia.org/male.S/s/sells-tommy-ly...,Lethal Injection,Tommy Lynn Sells,White
5,NOT FOUND,"October 16, 1984","January 16, 2003",NOT FOUND,"Sentenced to death on May 28, 2004. Executed b...",NOT FOUND,Robbery,NOT FOUND,1,http://murderpedia.org/male.D/d/doyle-anthony-...,Texas,Dallas,03/27/2014,http://murderpedia.org/male.D/d/doyle-anthony-...,Lethal Injection,Anthony Dewayne Doyle,Black
6,Same day,"August 25, 1980","November 29, 1998",NOT FOUND,"Sentenced to death on February 4, 2000. Execut...",NOT FOUND,Rapper - Robbery,NOT FOUND,1,http://murderpedia.org/male.J/j/jasper-ray.htm,Texas,Bexar,03/19/2014,http://murderpedia.org/male.J/j/jasper-ray.htm,Lethal Injection,Ray Jasper,Black
7,NOT FOUND,"May 15, 1954","August 25, 1998",NOT FOUND,"Sentenced to death on September 1, 1999. Execu...",NOT FOUND,Kidnapping - Torture - To collect insurance money,NOT FOUND,1,http://murderpedia.org/female.B/b/basso-suzann...,Texas,Harris,02/05/2014,http://murderpedia.org/female.B/b/basso-suzann...,Lethal Injection,Suzanne Margaret Basso,White
8,Same day,"July 22, 1967","January 31, 1994",NOT FOUND,"Sentenced to death on November 18, 1994. Execu...",NOT FOUND,Escape attempt,NOT FOUND,1,http://murderpedia.org/male.T/t/tamayo-edgar-a...,Texas,Harris,01/22/2014,http://murderpedia.org/male.T/t/tamayo-edgar-a...,Lethal Injection,Edgar Arias Tamayo,Hispanic
9,Same day,"March 28, 1970","September 24, 2007",NOT FOUND,"Sentenced to death on February 17, 2009. Execu...",NOT FOUND,Prison break,NOT FOUND,1,http://murderpedia.org/male.M/m1/martin-jerry-...,Texas,Walker,12/03/2013,http://murderpedia.org/male.M/m1/martin-jerry-...,Lethal Injection,Jerry Duane Martin,White


In [57]:
df_all_texas.to_csv('texas-2.csv', index=False)

In [59]:
df_tex = pd.read_csv('texas_2_bis.csv', encoding='latin1')

In [61]:
df_tex['year'] = df_all_texas['date_execution'].str.extract(r"(\d\d\d\d)")

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [65]:
df_tex['date_exe'] = df_all_texas['date_execution']
df_tex = df_tex.fillna('Unknown')
df_tex['date_offense'] = df_all_texas['date_offense']
df_tex

,date_arrest,date_of_birth,date_offense,final_words,method_x,mitigating_circumstances,offense,race_x,victim_num,url,State,county,date_execution,link_to_profile,method_y,name,race_y,gender,year,date_exe
0,NOT FOUND,6-Oct-75,"July 26, 2004","""I'm all right. Tell them I finished strong. G...","Sentenced to death on June 22, 2006. Executed ...",NOT FOUND,Child abuse - Torture,NOT FOUND,1,http://murderpedia.org/female.C/c/coleman-lisa...,Texas,Tarrant,9/17/14,http://murderpedia.org/female.C/c/coleman-lisa...,Lethal Injection,Lisa Ann Coleman,Black,Female,2014,09/17/2014
1,Same day,8-Sep-69,"May 3, 1993","""I love you all. I'm going home, going to be w...","Sentenced to death on December 15, 1993. Execu...",NOT FOUND,Shooting rampage,NOT FOUND,2,http://murderpedia.org/male.T/t/trottie-willie...,Texas,Harris,9/10/14,http://murderpedia.org/male.T/t/trottie-willie...,Lethal Injection,Willie Tyrone Trottie,Black,Male,2014,09/10/2014
2,Same day,14-Apr-75,"January 22, 2001","""I would like to remind my children once again...","Sentenced to death on May 21, 2002. Executed b...",NOT FOUND,Parricide - Drugs - Robbery,NOT FOUND,3,http://murderpedia.org/male.V/v/villegas-jose-...,Texas,Nueces,4/16/14,http://murderpedia.org/male.V/v/villegas-jose-...,Lethal Injection,Jose Luis Villegas Jr.,Hispanic,Male,2014,04/16/2014
3,Same day,5-Oct-69,1997 / 1999,"""I'm happy,"" Hernandez said in Spanish. ""I am ...","Sentenced to death on February 11, 2000. Execu...",NOT FOUND,Rape,NOT FOUND,2,http://murderpedia.org/male.H/h/hernandez-rami...,Texas,Bandera,4/9/14,http://murderpedia.org/male.H/h/hernandez-rami...,Lethal Injection,Ramiro Hernandez,Hispanic,Male,2014,04/09/2014
4,2-Jan-00,28-Jun-64,1980 - 1999,None.,"Sentenced to death in Texas on November 8, 200...",NOT FOUND,Rape,NOT FOUND,1 - 13 +,http://murderpedia.org/male.S/s/sells-tommy-ly...,Texas,Val Verde,4/3/14,http://murderpedia.org/male.S/s/sells-tommy-ly...,Lethal Injection,Tommy Lynn Sells,White,Male,2014,04/03/2014
5,NOT FOUND,16-Oct-84,"January 16, 2003",None.,"Sentenced to death on May 28, 2004. Executed b...",NOT FOUND,Robbery,NOT FOUND,1,http://murderpedia.org/male.D/d/doyle-anthony-...,Texas,Dallas,3/27/14,http://murderpedia.org/male.D/d/doyle-anthony-...,Lethal Injection,Anthony Dewayne Doyle,Black,Male,2014,03/27/2014
6,Same day,25-Aug-80,"November 29, 1998","Before being put to death, Jasper spoke quietl...","Sentenced to death on February 4, 2000. Execut...",NOT FOUND,Rapper - Robbery,NOT FOUND,1,http://murderpedia.org/male.J/j/jasper-ray.htm,Texas,Bexar,3/19/14,http://murderpedia.org/male.J/j/jasper-ray.htm,Lethal Injection,Ray Jasper,Black,Male,2014,03/19/2014
7,NOT FOUND,15-May-54,"August 25, 1998","Before being put to death, Basso told a warden...","Sentenced to death on September 1, 1999. Execu...",NOT FOUND,Kidnapping - Torture - To collect insurance money,NOT FOUND,1,http://murderpedia.org/female.B/b/basso-suzann...,Texas,Harris,2/5/14,http://murderpedia.org/female.B/b/basso-suzann...,Lethal Injection,Suzanne Margaret Basso,White,Female,2014,02/05/2014
8,Same day,22-Jul-67,"January 31, 1994","Tamayo made no final statement, but earlier We...","Sentenced to death on November 18, 1994. Execu...",NOT FOUND,Escape attempt,NOT FOUND,1,http://murderpedia.org/male.T/t/tamayo-edgar-a...,Texas,Harris,1/22/14,http://murderpedia.org/male.T/t/tamayo-edgar-a...,Lethal Injection,Edgar Arias Tamayo,Hispanic,Male,2014,01/22/2014
9,Same day,28-Mar-70,"September 24, 2007","""I did not murder your loved one. It was an ac...","Sentenced to death on February 17, 2009. Execu...",NOT FOUND,Prison break,NOT FOUND,1,http://murderpedia.org/male.M/m1/martin-jerry-...,Texas,Walker,12/3/13,http://murderpedia.org/male.M/m1/martin-jerry-...,Lethal Injection,Jerry Duane Martin,White,Male,2013,12/03/2013


In [66]:
df_tex.to_csv('final_texas.csv', index=False)

# Arizona

In [68]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/arizona.htm')

In [69]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Arizona"
    rows_df.append(row_df)
#print(rows_df)
df_arizona = pd.DataFrame(rows_df)
df_arizona

,State,county,date_execution,link_to_profile,method,name
0,Arizona,Pima,"July 23, 2014",http://murderpedia.org/male.W/w/wood-joseph.htm,Lethal Injection,Joseph Rudolph Wood III
1,Arizona,Pima,"October 23, 2013",http://murderpedia.org/male.J/j/jones-robert-g...,Lethal Injection,Robert Glen Jones
2,Arizona,Yavapai,"October 9, 2013",http://murderpedia.org/male.S/s/schad-edward.htm,Lethal Injection,Edward Harold Schad Jr.
3,Arizona,Cochise,"December 5, 2012",http://murderpedia.org/male.S/s/stokley-richar...,Lethal Injection,Richard Dale Stokley
4,Arizona,Mohave,"August 8, 2012",http://murderpedia.org/male.C/c/cook-daniel-wa...,Lethal Injection,Daniel Wayne Cook
5,Arizona,Maricopa,"June 27, 2012",http://murderpedia.org/male.L/l/lopez-samuel-v...,Lethal Injection,Samuel Villegas Lopez
6,Arizona,Pima,"April 25, 2012",http://murderpedia.org/male.K/k/kemp-thomas.htm,Lethal Injection,Thomas Arnold Kemp Jr.
7,Arizona,Maricopa,"March 8, 2012",http://murderpedia.org/male.T/t/towery-robert.htm,Lethal Injection,Robert Charles Towery
8,Arizona,Pinal,"February 29, 2012",http://murderpedia.org/male.M/m1/moorman-rober...,Lethal Injection,Robert Henry Moormann
9,Arizona,Pima,"July 19, 2011",http://murderpedia.org/male.W/w1/west-thomas-p...,Lethal Injection,Thomas Paul West


In [70]:
def scrape_each_execution_arizona(row):
    driver.get(row['link_to_profile'])
    try:
        table= driver.find_element_by_id('AutoNumber3')
    except:
        pass
    try:
        table = driver.find_elements_by_tag_name('table')[13]
    except:
        pass
    
    row = table.find_elements_by_tag_name('tr')[0].text

    convict = {}
    try:
        offense = re.findall(r"Characteristics: (.*)", row)[0]
    except:
        offense = 'NOT FOUND'
    try:
        date_offense = re.findall(r"Date of murders?: (.*)", row)[0]
    except:
        date_offense = 'NOT FOUND'
    try:
        date_arrest = re.findall(r"Date of arrest: (.*)", row)[0]
    except:
        date_arrest = 'NOT FOUND'
    try:
        date_of_birth = re.findall(r"Date of birth: (.*)", row)[0]
    except:
        date_of_birth = 'NOT FOUND'
    try:
        victim_num = re.findall(r"Number of victims: (.*)", row)[0]
    except:
        victim_num = 'NOT FOUND'
    try:
        method = re.findall(r"Status: (.*)", row)[0]
    except:
        method = 'NOT FOUND'
    try:
        race = re.findall(r"DOC#: \d*[\n\s](.*)\n", row)[0]
    except:
        race = 'NOT FOUND'
    try:
        final_words = re.findall(r"Words:\n\n(.*)\n", row)[0]
    except:
        final_words = 'NOT FOUND'
    try:
        mitigating_circumstances = re.findall(r"Mitigating Circumstances: (.*)", row)[0]
    except:
        mitigating_circumstances = 'NOT FOUND'
    return pd.Series({
         'offense': offense,
        'date_offense': date_offense,
        'date_arrest':date_arrest,
     'date_of_birth':date_of_birth,
     'victim_num':victim_num,
     'method':method,
    'race': race,
    'final_words': final_words,
    'mitigating_circumstances' : mitigating_circumstances
     })

In [71]:
df_all_arizona = df_arizona.apply(scrape_each_execution_arizona, axis=1)
df_all_arizona

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,Same day (wounded by police),"December 6, 1958","August 7, 1989",Wood looked at the victim's family as he deliv...,"Sentenced to death on July 2, 1991. Executed b...",NOT FOUND,Revenge - Domestic violence,NOT FOUND,2
1,NOT FOUND,"December 25, 1969",May-August 1996,NOT FOUND,"Sentenced to death on February 17, 2000. Execu...",,Robberies,NOT FOUND,7
2,"September 9, 1978","July 27, 1942","August 1, 1978",NOT FOUND,"Sentenced to death on December 27, 1979. Resen...",NOT FOUND,Robbery,NOT FOUND,1
3,Next day,"September 9, 1952","July 8, 1991",None.,"Sentenced to death on July 14, 1992. Executed ...",NOT FOUND,Rape,DOB: 09/09/1952,2
4,2 days after,"July 23, 1961","July 19, 1987","""I'd like to say sorry to the victim's family....","Sentenced to death on August 8, 1988. Executed...",None,Homosexual rapist - Torture murders,DOB: 07/23/61,2
5,NOT FOUND,"June 30, 1962","October 29, 1986",None.,"Sentenced to death on June 25, 1987. Executed ...",NOT FOUND,Rape,DOB: 06/30/1962,1
6,NOT FOUND,"June 2, 1948","July 11, 1992","""I regret nothing.""","Sentenced to death on July 9, 1993. Executed b...",NOT FOUND,Robbery,DOB: 06/02/1948,1
7,NOT FOUND,"July 20, 1964","September 4, 1991",NOT FOUND,"Sentenced to death on November 20, 1992. Execu...",NOT FOUND,Robbery,NOT FOUND,1
8,Next day,"June 4, 1948","January 13, 1984",NOT FOUND,"Sentenced to death on May 7, 1985. Executed by...",NOT FOUND,Parricide - Incest - Dismemberment,NOT FOUND,1
9,NOT FOUND,"April 28, 1959","July 12, 1987",None.,"Sentenced to death on August 1, 1988. Executed...",NOT FOUND,Robbery,DOB: 04/28/1959,1


In [73]:
df_all_arizona['url'] = df_arizona.link_to_profile 
df_all_arizona = df_all_arizona.merge(df_arizona, right_on='link_to_profile', left_on = 'url')
df_all_arizona

,date_arrest,date_of_birth,date_offense,final_words,method_x,mitigating_circumstances,offense,race,victim_num,url,State,county,date_execution,link_to_profile,method_y,name
0,Same day (wounded by police),"December 6, 1958","August 7, 1989",Wood looked at the victim's family as he deliv...,"Sentenced to death on July 2, 1991. Executed b...",NOT FOUND,Revenge - Domestic violence,NOT FOUND,2,http://murderpedia.org/male.W/w/wood-joseph.htm,Arizona,Pima,"July 23, 2014",http://murderpedia.org/male.W/w/wood-joseph.htm,Lethal Injection,Joseph Rudolph Wood III
1,NOT FOUND,"December 25, 1969",May-August 1996,NOT FOUND,"Sentenced to death on February 17, 2000. Execu...",,Robberies,NOT FOUND,7,http://murderpedia.org/male.J/j/jones-robert-g...,Arizona,Pima,"October 23, 2013",http://murderpedia.org/male.J/j/jones-robert-g...,Lethal Injection,Robert Glen Jones
2,"September 9, 1978","July 27, 1942","August 1, 1978",NOT FOUND,"Sentenced to death on December 27, 1979. Resen...",NOT FOUND,Robbery,NOT FOUND,1,http://murderpedia.org/male.S/s/schad-edward.htm,Arizona,Yavapai,"October 9, 2013",http://murderpedia.org/male.S/s/schad-edward.htm,Lethal Injection,Edward Harold Schad Jr.
3,Next day,"September 9, 1952","July 8, 1991",None.,"Sentenced to death on July 14, 1992. Executed ...",NOT FOUND,Rape,DOB: 09/09/1952,2,http://murderpedia.org/male.S/s/stokley-richar...,Arizona,Cochise,"December 5, 2012",http://murderpedia.org/male.S/s/stokley-richar...,Lethal Injection,Richard Dale Stokley
4,2 days after,"July 23, 1961","July 19, 1987","""I'd like to say sorry to the victim's family....","Sentenced to death on August 8, 1988. Executed...",None,Homosexual rapist - Torture murders,DOB: 07/23/61,2,http://murderpedia.org/male.C/c/cook-daniel-wa...,Arizona,Mohave,"August 8, 2012",http://murderpedia.org/male.C/c/cook-daniel-wa...,Lethal Injection,Daniel Wayne Cook
5,NOT FOUND,"June 30, 1962","October 29, 1986",None.,"Sentenced to death on June 25, 1987. Executed ...",NOT FOUND,Rape,DOB: 06/30/1962,1,http://murderpedia.org/male.L/l/lopez-samuel-v...,Arizona,Maricopa,"June 27, 2012",http://murderpedia.org/male.L/l/lopez-samuel-v...,Lethal Injection,Samuel Villegas Lopez
6,NOT FOUND,"June 2, 1948","July 11, 1992","""I regret nothing.""","Sentenced to death on July 9, 1993. Executed b...",NOT FOUND,Robbery,DOB: 06/02/1948,1,http://murderpedia.org/male.K/k/kemp-thomas.htm,Arizona,Pima,"April 25, 2012",http://murderpedia.org/male.K/k/kemp-thomas.htm,Lethal Injection,Thomas Arnold Kemp Jr.
7,NOT FOUND,"July 20, 1964","September 4, 1991",NOT FOUND,"Sentenced to death on November 20, 1992. Execu...",NOT FOUND,Robbery,NOT FOUND,1,http://murderpedia.org/male.T/t/towery-robert.htm,Arizona,Maricopa,"March 8, 2012",http://murderpedia.org/male.T/t/towery-robert.htm,Lethal Injection,Robert Charles Towery
8,Next day,"June 4, 1948","January 13, 1984",NOT FOUND,"Sentenced to death on May 7, 1985. Executed by...",NOT FOUND,Parricide - Incest - Dismemberment,NOT FOUND,1,http://murderpedia.org/male.M/m1/moorman-rober...,Arizona,Pinal,"February 29, 2012",http://murderpedia.org/male.M/m1/moorman-rober...,Lethal Injection,Robert Henry Moormann
9,NOT FOUND,"April 28, 1959","July 12, 1987",None.,"Sentenced to death on August 1, 1988. Executed...",NOT FOUND,Robbery,DOB: 04/28/1959,1,http://murderpedia.org/male.W/w1/west-thomas-p...,Arizona,Pima,"July 19, 2011",http://murderpedia.org/male.W/w1/west-thomas-p...,Lethal Injection,Thomas Paul West


In [74]:
df_all_arizona.to_csv('arizona-2.csv', index=False)

In [86]:
df_ari = pd.read_csv('arizona-2.csv', encoding='latin1', na_values='NOT FOUND')

In [87]:
df_ari['year'] = df_all_arizona['date_execution'].str.extract(r"(\d\d\d\d)")
df_ari['date_exe'] = df_all_arizona['date_execution']
df_ari = df_ari.fillna('Unknown')
df_ari['date_offense'] = df_all_arizona['date_offense']

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [90]:
df_ari.to_csv('final_arizona.csv', index=False)

In [89]:
df_ari

,date_arrest,date_of_birth,date_offense,final_words,method_x,mitigating_circumstances,offense,race,victim_num,url,State,county,date_execution,link_to_profile,method_y,name,race.1,gender,year,date_exe
0,Same day (wounded by police),6-Dec-58,"August 7, 1989",Wood looked at the victim's family as he deliv...,"Sentenced to death on July 2, 1991. Executed b...",Unknown,Revenge - Domestic violence,Unknown,2,http://murderpedia.org/male.W/w/wood-joseph.htm,Arizona,Pima,23-Jul-14,http://murderpedia.org/male.W/w/wood-joseph.htm,Lethal Injection,Joseph Rudolph Wood III,White,Male,2014,"July 23, 2014"
1,Unknown,25-Dec-69,May-August 1996,"""Love and respect my family and friends and I ...","Sentenced to death on February 17, 2000. Execu...",Unknown,Robberies,Unknown,7,http://murderpedia.org/male.J/j/jones-robert-g...,Arizona,Pima,23-Oct-13,http://murderpedia.org/male.J/j/jones-robert-g...,Lethal Injection,Robert Glen Jones,White,Male,2013,"October 23, 2013"
2,9-Sep-78,27-Jul-42,"August 1, 1978","""Well, after 34 years, I'm free to fly away ho...","Sentenced to death on December 27, 1979. Resen...",Unknown,Robbery,Unknown,1,http://murderpedia.org/male.S/s/schad-edward.htm,Arizona,Yavapai,9-Oct-13,http://murderpedia.org/male.S/s/schad-edward.htm,Lethal Injection,Edward Harold Schad Jr.,White,Male,2013,"October 9, 2013"
3,Next day,9-Sep-52,"July 8, 1991",None.,"Sentenced to death on July 14, 1992. Executed ...",Unknown,Rape,DOB: 09/09/1952,2,http://murderpedia.org/male.S/s/stokley-richar...,Arizona,Cochise,5-Dec-12,http://murderpedia.org/male.S/s/stokley-richar...,Lethal Injection,Richard Dale Stokley,White,Male,2012,"December 5, 2012"
4,2 days after,23-Jul-61,"July 19, 1987","""I'd like to say sorry to the victim's family....","Sentenced to death on August 8, 1988. Executed...",None,Homosexual rapist - Torture murders,DOB: 07/23/61,2,http://murderpedia.org/male.C/c/cook-daniel-wa...,Arizona,Mohave,8-Aug-12,http://murderpedia.org/male.C/c/cook-daniel-wa...,Lethal Injection,Daniel Wayne Cook,White,Male,2012,"August 8, 2012"
5,Unknown,30-Jun-62,"October 29, 1986",None.,"Sentenced to death on June 25, 1987. Executed ...",Unknown,Rape,DOB: 06/30/1962,1,http://murderpedia.org/male.L/l/lopez-samuel-v...,Arizona,Maricopa,27-Jun-12,http://murderpedia.org/male.L/l/lopez-samuel-v...,Lethal Injection,Samuel Villegas Lopez,Hispanic,Male,2012,"June 27, 2012"
6,Unknown,2-Jun-48,"July 11, 1992","""I regret nothing.""","Sentenced to death on July 9, 1993. Executed b...",Unknown,Robbery,DOB: 06/02/1948,1,http://murderpedia.org/male.K/k/kemp-thomas.htm,Arizona,Pima,25-Apr-12,http://murderpedia.org/male.K/k/kemp-thomas.htm,Lethal Injection,Thomas Arnold Kemp Jr.,White,Male,2012,"April 25, 2012"
7,Unknown,20-Jul-64,"September 4, 1991","In his final words before being put to death, ...","Sentenced to death on November 20, 1992. Execu...",Unknown,Robbery,Unknown,1,http://murderpedia.org/male.T/t/towery-robert.htm,Arizona,Maricopa,8-Mar-12,http://murderpedia.org/male.T/t/towery-robert.htm,Lethal Injection,Robert Charles Towery,White,Male,2012,"March 8, 2012"
8,Next day,4-Jun-48,"January 13, 1984","""I hope that this will bring closure and they ...","Sentenced to death on May 7, 1985. Executed by...",Unknown,Parricide - Incest - Dismemberment,Unknown,1,http://murderpedia.org/male.M/m1/moorman-rober...,Arizona,Pinal,29-Feb-12,http://murderpedia.org/male.M/m1/moorman-rober...,Lethal Injection,Robert Henry Moormann,White,Male,2012,"February 29, 2012"
9,Unknown,28-Apr-59,"July 12, 1987",None.,"Sentenced to death on August 1, 1988. Executed...",Unknown,Robbery,DOB: 04/28/1959,1,http://murderpedia.org/male.W/w1/west-thomas-p...,Arizona,Pima,19-Jul-11,http://murderpedia.org/male.W/w1/west-thomas-p...,Lethal Injection,Thomas Paul West,White,Male,2011,"July 19, 2011"


# Arkansas

In [96]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/arkansas.htm')

In [92]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Arkansas"
    rows_df.append(row_df)
#print(rows_df)
df_arkansas = pd.DataFrame(rows_df)
df_arkansas

,State,county,date_execution,link_to_profile,method,name
0,Arkansas,Hot Spring,"November 28, 2005",http://murderpedia.org/male.N/n1/nance-eric.htm,Lethal Injection,Eric Randall Nance
1,Arkansas,Ashley,"January 6, 2004",http://murderpedia.org/male.S/s1/singleton-cha...,Lethal Injection,Charles Laverne Singleton
2,Arkansas,Pulaski,"July 9, 2003",http://murderpedia.org/male.N/n1/noel-riley.htm,Lethal Injection,Riley Dobi Noel
3,Arkansas,Jefferson,"May 8, 2001",http://murderpedia.org/male.S/s1/smith-clay-ki...,Lethal Injection,Clay King Smith
4,Arkansas,Pulaski,"December 19, 2000",http://murderpedia.org/male.J/j1/johnson-david...,Lethal Injection,David Dewayne Johnson
5,Arkansas,Pulaski,"May 2, 2000",http://deathpenaltyusa.org/usa/female/riggs-ch...,Lethal Injection,Christina Marie Riggs
6,Arkansas,Johnson,"September 8, 1999",http://murderpedia.org/male.W/w1/willet-alan.htm,Lethal Injection,Allan Willet
7,Arkansas,Sebastian,"September 8, 1999",http://murderpedia.org/male.G/g1/gardner-mark-...,Lethal Injection,Mark Edward Gardner
8,Arkansas,Sebastian,"April 12, 1999",http://murderpedia.org/male.P/p1/pruett-marion...,Lethal Injection,Marion Albert Pruett
9,Arkansas,White,"February 16, 1999",http://murderpedia.org/male.C/c1/cox-john-mich...,Lethal Injection,John Michael Cox


In [145]:
def scrape_each_execution_arkansas(row):
    driver.get(row['link_to_profile'])
    try:
        table= driver.find_element_by_id('AutoNumber3')
    except:
        pass
    try:
        table = driver.find_elements_by_tag_name('table')[13]
    except:
        pass
    
    try:
        row = table.find_elements_by_tag_name('tr')[0].text

        convict = {}
        try:
            offense = re.findall(r"Characteristics: (.*)", row)[0]
        except:
            offense = 'NOT FOUND'
        try:
            date_offense = re.findall(r"Date of murders?: (.*)", row)[0]
        except:
            date_offense = 'NOT FOUND'
        try:
            date_arrest = re.findall(r"Date of arrest: (.*)", row)[0]
        except:
            date_arrest = 'NOT FOUND'
        try:
            date_of_birth = re.findall(r"Date of birth: (.*)", row)[0]
        except:
            date_of_birth = 'NOT FOUND'
        try:
            victim_num = re.findall(r"Number of victims: (.*)", row)[0]
        except:
            victim_num = 'NOT FOUND'
        try:
            method = re.findall(r"Status: (.*)", row)[0]
        except:
            method = 'NOT FOUND'
        try:
            race = re.findall(r"Race: (.*)\n", row)[0]
        except:
            race = 'NOT FOUND'
        try:
            final_words = re.findall(r"Words:\n\n(.*)\n", row)[0]
        except:
            final_words = 'NOT FOUND'
        try:
            mitigating_circumstances = re.findall(r"Mitigating Circumstances: (.*)", row)[0]
        except:
            mitigating_circumstances = 'NOT FOUND'
        return pd.Series({
             'offense': offense,
            'date_offense': date_offense,
            'date_arrest':date_arrest,
         'date_of_birth':date_of_birth,
         'victim_num':victim_num,
         'method':method,
        'race': race,
        'final_words': final_words,
        'mitigating_circumstances' : mitigating_circumstances
         })
    
    except: return pd.Series({
             'offense': "NOT FOUND",
            'date_offense': "NOT FOUND",
            'date_arrest':"NOT FOUND",
         'date_of_birth':"NOT FOUND",
         'victim_num':"NOT FOUND",
         'method':"NOT FOUND",
        'race': "NOT FOUND",
        'final_words': "NOT FOUND",
        'mitigating_circumstances' :"NOT FOUND"
         })
    

In [97]:
df_all_arkansas = df_arkansas.apply(scrape_each_execution_arkansas, axis=1)
df_all_arkansas

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,9 days after,"January 9, 1960","October 11, 1993",None.,Executed by lethal injection in Arkansas on No...,NOT FOUND,Rape,WHITE,1
1,NOT FOUND,"March 29, 1959","June 1, 1979",His final statement was released in written fo...,Executed by lethal injection in Arkansas on Ja...,NOT FOUND,Diagnosed schizophrenic - Robbery,BLACK,1
2,4 days after,"May 22, 1972","June 4, 1995","""I want my family to know I love them. I want ...",Executed by lethal injection in Arkansas on Ju...,NOT FOUND,Revenge,BLACK,3
3,Same day (wounded by police),"July 25, 1970","March 25, 1998",NOT FOUND,Executed by lethal injection in Arkansas on Ma...,NOT FOUND,Maltreatment - Revenge,NOT FOUND,5
4,Same day,"January 10, 1963","September 2, 1989",NOT FOUND,Executed by lethal injection in Arkansas on De...,NOT FOUND,Robbery,NOT FOUND,1
5,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND
6,Same day (suicide attempt),1947,"September 14, 1993",NOT FOUND,Executed by lethal injection in Arkansas on Se...,NOT FOUND,Parricide,NOT FOUND,2
7,NOT FOUND,1956,"December 12, 1985",NOT FOUND,Executed by lethal injection in Arkansas on Se...,NOT FOUND,Rape - Robbery,NOT FOUND,3
8,NOT FOUND,1949,1981,NOT FOUND,Executed by lethal injection in Arkansas on Ap...,NOT FOUND,Robberies - Drugs,NOT FOUND,5 +
9,NOT FOUND,1947,"November 1, 1989",NOT FOUND,Executed by lethal injection in Arkansas on Fe...,NOT FOUND,Arson,NOT FOUND,3


In [98]:
df_all_arkansas['url'] = df_arkansas.link_to_profile 
df_all_arkansas = df_all_arkansas.merge(df_arkansas, right_on='link_to_profile', left_on = 'url')
df_all_arkansas

,date_arrest,date_of_birth,date_offense,final_words,method_x,mitigating_circumstances,offense,race,victim_num,url,State,county,date_execution,link_to_profile,method_y,name
0,9 days after,"January 9, 1960","October 11, 1993",None.,Executed by lethal injection in Arkansas on No...,NOT FOUND,Rape,WHITE,1,http://murderpedia.org/male.N/n1/nance-eric.htm,Arkansas,Hot Spring,"November 28, 2005",http://murderpedia.org/male.N/n1/nance-eric.htm,Lethal Injection,Eric Randall Nance
1,NOT FOUND,"March 29, 1959","June 1, 1979",His final statement was released in written fo...,Executed by lethal injection in Arkansas on Ja...,NOT FOUND,Diagnosed schizophrenic - Robbery,BLACK,1,http://murderpedia.org/male.S/s1/singleton-cha...,Arkansas,Ashley,"January 6, 2004",http://murderpedia.org/male.S/s1/singleton-cha...,Lethal Injection,Charles Laverne Singleton
2,4 days after,"May 22, 1972","June 4, 1995","""I want my family to know I love them. I want ...",Executed by lethal injection in Arkansas on Ju...,NOT FOUND,Revenge,BLACK,3,http://murderpedia.org/male.N/n1/noel-riley.htm,Arkansas,Pulaski,"July 9, 2003",http://murderpedia.org/male.N/n1/noel-riley.htm,Lethal Injection,Riley Dobi Noel
3,Same day (wounded by police),"July 25, 1970","March 25, 1998",NOT FOUND,Executed by lethal injection in Arkansas on Ma...,NOT FOUND,Maltreatment - Revenge,NOT FOUND,5,http://murderpedia.org/male.S/s1/smith-clay-ki...,Arkansas,Jefferson,"May 8, 2001",http://murderpedia.org/male.S/s1/smith-clay-ki...,Lethal Injection,Clay King Smith
4,Same day,"January 10, 1963","September 2, 1989",NOT FOUND,Executed by lethal injection in Arkansas on De...,NOT FOUND,Robbery,NOT FOUND,1,http://murderpedia.org/male.J/j1/johnson-david...,Arkansas,Pulaski,"December 19, 2000",http://murderpedia.org/male.J/j1/johnson-david...,Lethal Injection,David Dewayne Johnson
5,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,http://deathpenaltyusa.org/usa/female/riggs-ch...,Arkansas,Pulaski,"May 2, 2000",http://deathpenaltyusa.org/usa/female/riggs-ch...,Lethal Injection,Christina Marie Riggs
6,Same day (suicide attempt),1947,"September 14, 1993",NOT FOUND,Executed by lethal injection in Arkansas on Se...,NOT FOUND,Parricide,NOT FOUND,2,http://murderpedia.org/male.W/w1/willet-alan.htm,Arkansas,Johnson,"September 8, 1999",http://murderpedia.org/male.W/w1/willet-alan.htm,Lethal Injection,Allan Willet
7,NOT FOUND,1956,"December 12, 1985",NOT FOUND,Executed by lethal injection in Arkansas on Se...,NOT FOUND,Rape - Robbery,NOT FOUND,3,http://murderpedia.org/male.G/g1/gardner-mark-...,Arkansas,Sebastian,"September 8, 1999",http://murderpedia.org/male.G/g1/gardner-mark-...,Lethal Injection,Mark Edward Gardner
8,NOT FOUND,1949,1981,NOT FOUND,Executed by lethal injection in Arkansas on Ap...,NOT FOUND,Robberies - Drugs,NOT FOUND,5 +,http://murderpedia.org/male.P/p1/pruett-marion...,Arkansas,Sebastian,"April 12, 1999",http://murderpedia.org/male.P/p1/pruett-marion...,Lethal Injection,Marion Albert Pruett
9,NOT FOUND,1947,"November 1, 1989",NOT FOUND,Executed by lethal injection in Arkansas on Fe...,NOT FOUND,Arson,NOT FOUND,3,http://murderpedia.org/male.C/c1/cox-john-mich...,Arkansas,White,"February 16, 1999",http://murderpedia.org/male.C/c1/cox-john-mich...,Lethal Injection,John Michael Cox


In [101]:
df_all_arkansas.to_csv('arkansas-2.csv', index=False)

In [108]:
df_ark = pd.read_csv('arkansas-2-here.csv', encoding='latin1', na_values='NOT FOUND')


In [109]:
df_ark

,date_arrest,date_of_birth,date_offense,final_words,method_x,mitigating_circumstances,offense,race,victim_num,url,State,county,date_execution,link_to_profile,method_y,name,race_,gender
0,NaN,NaN,13-Dec-98,"""I humbly extend my sincerest of apologies to ...",Executed by lethal injection,NaN,NaN,NaN,1,NaN,Arkansas,Lincoln,27-Apr-17,NaN,Lethal Injection,Kenneth Williams,Black,Male
1,NaN,NaN,6-Jun-95,"He apologized to the girl he nearly killed. ""I...",Executed by lethal injection,NaN,NaN,NaN,1,NaN,Arkansas,White,24-Apr-17,NaN,Lethal Injection,Jack Jones,White,Male
2,NaN,NaN,20-Nov-94,None.,Executed by lethal injection,NaN,NaN,NaN,1,NaN,Arkansas,Pulaski,24-Apr-17,NaN,Lethal Injection,Marcel Williams,Black,Male
3,NaN,31-Jul-65,9-Feb-95,None.,Executed by lethal injection,NaN,NaN,NaN,1,NaN,Arkansas,Pulaski,20-Apr-17,NaN,Lethal Injection,Ledell Lee,Black,Male
4,9 days after,9-Jan-60,11-Oct-93,None.,Executed by lethal injection in Arkansas on No...,NaN,Rape,WHITE,1,http://murderpedia.org/male.N/n1/nance-eric.htm,Arkansas,Hot Spring,28-Nov-05,http://murderpedia.org/male.N/n1/nance-eric.htm,Lethal Injection,Eric Randall Nance,White,Male
5,NaN,29-Mar-59,1-Jun-79,His final statement was released in written fo...,Executed by lethal injection in Arkansas on Ja...,NaN,Diagnosed schizophrenic - Robbery,BLACK,1,http://murderpedia.org/male.S/s1/singleton-cha...,Arkansas,Ashley,6-Jan-04,http://murderpedia.org/male.S/s1/singleton-cha...,Lethal Injection,Charles Laverne Singleton,Black,Male
6,4 days after,22-May-72,4-Jun-95,"""I want my family to know I love them. I want ...",Executed by lethal injection in Arkansas on Ju...,NaN,Revenge,BLACK,3,http://murderpedia.org/male.N/n1/noel-riley.htm,Arkansas,Pulaski,9-Jul-03,http://murderpedia.org/male.N/n1/noel-riley.htm,Lethal Injection,Riley Dobi Noel,Black,Male
7,Same day (wounded by police),25-Jul-70,25-Mar-98,"""I'd like to say I'm sorry about what I did to...",Executed by lethal injection in Arkansas on Ma...,NaN,Maltreatment - Revenge,NaN,5,http://murderpedia.org/male.S/s1/smith-clay-ki...,Arkansas,Jefferson,8-May-01,http://murderpedia.org/male.S/s1/smith-clay-ki...,Lethal Injection,Clay King Smith,White,Male
8,Same day,10-Jan-63,2-Sep-89,None.,Executed by lethal injection in Arkansas on De...,NaN,Robbery,NaN,1,http://murderpedia.org/male.J/j1/johnson-david...,Arkansas,Pulaski,19-Dec-00,http://murderpedia.org/male.J/j1/johnson-david...,Lethal Injection,David Dewayne Johnson,Black,Male
9,Next day,1971,4-Nov-97,"""There is no way no words can express how sorr...",NaN,NaN,NaN,NaN,NaN,http://deathpenaltyusa.org/usa/female/riggs-ch...,Arkansas,Pulaski,2-May-00,http://deathpenaltyusa.org/usa/female/riggs-ch...,Lethal Injection,Christina Marie Riggs,White,Female


In [112]:
df_ark['year'] = df_all_arkansas['date_execution'].str.extract(r"(\d\d\d\d)")
df_ark['date_exe'] = df_all_arkansas['date_execution']
df_ark = df_ark.fillna('Unknown')
df_ark['date_offense'] = df_all_arkansas['date_offense']

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [111]:
df_ark.to_csv('final_arkansas.csv', index=False)

In [113]:
df_ark

,date_arrest,date_of_birth,date_offense,final_words,method_x,mitigating_circumstances,offense,race,victim_num,url,State,county,date_execution,link_to_profile,method_y,name,race_,gender,year,date_exe
0,Unknown,Unknown,"October 11, 1993","""I humbly extend my sincerest of apologies to ...",Executed by lethal injection,Unknown,Unknown,Unknown,1,Unknown,Arkansas,Lincoln,27-Apr-17,Unknown,Lethal Injection,Kenneth Williams,Black,Male,2005,"November 28, 2005"
1,Unknown,Unknown,"June 1, 1979","He apologized to the girl he nearly killed. ""I...",Executed by lethal injection,Unknown,Unknown,Unknown,1,Unknown,Arkansas,White,24-Apr-17,Unknown,Lethal Injection,Jack Jones,White,Male,2004,"January 6, 2004"
2,Unknown,Unknown,"June 4, 1995",None.,Executed by lethal injection,Unknown,Unknown,Unknown,1,Unknown,Arkansas,Pulaski,24-Apr-17,Unknown,Lethal Injection,Marcel Williams,Black,Male,2003,"July 9, 2003"
3,Unknown,31-Jul-65,"March 25, 1998",None.,Executed by lethal injection,Unknown,Unknown,Unknown,1,Unknown,Arkansas,Pulaski,20-Apr-17,Unknown,Lethal Injection,Ledell Lee,Black,Male,2001,"May 8, 2001"
4,9 days after,9-Jan-60,"September 2, 1989",None.,Executed by lethal injection in Arkansas on No...,Unknown,Rape,WHITE,1,http://murderpedia.org/male.N/n1/nance-eric.htm,Arkansas,Hot Spring,28-Nov-05,http://murderpedia.org/male.N/n1/nance-eric.htm,Lethal Injection,Eric Randall Nance,White,Male,2000,"December 19, 2000"
5,Unknown,29-Mar-59,NOT FOUND,His final statement was released in written fo...,Executed by lethal injection in Arkansas on Ja...,Unknown,Diagnosed schizophrenic - Robbery,BLACK,1,http://murderpedia.org/male.S/s1/singleton-cha...,Arkansas,Ashley,6-Jan-04,http://murderpedia.org/male.S/s1/singleton-cha...,Lethal Injection,Charles Laverne Singleton,Black,Male,2000,"May 2, 2000"
6,4 days after,22-May-72,"September 14, 1993","""I want my family to know I love them. I want ...",Executed by lethal injection in Arkansas on Ju...,Unknown,Revenge,BLACK,3,http://murderpedia.org/male.N/n1/noel-riley.htm,Arkansas,Pulaski,9-Jul-03,http://murderpedia.org/male.N/n1/noel-riley.htm,Lethal Injection,Riley Dobi Noel,Black,Male,1999,"September 8, 1999"
7,Same day (wounded by police),25-Jul-70,"December 12, 1985","""I'd like to say I'm sorry about what I did to...",Executed by lethal injection in Arkansas on Ma...,Unknown,Maltreatment - Revenge,Unknown,5,http://murderpedia.org/male.S/s1/smith-clay-ki...,Arkansas,Jefferson,8-May-01,http://murderpedia.org/male.S/s1/smith-clay-ki...,Lethal Injection,Clay King Smith,White,Male,1999,"September 8, 1999"
8,Same day,10-Jan-63,1981,None.,Executed by lethal injection in Arkansas on De...,Unknown,Robbery,Unknown,1,http://murderpedia.org/male.J/j1/johnson-david...,Arkansas,Pulaski,19-Dec-00,http://murderpedia.org/male.J/j1/johnson-david...,Lethal Injection,David Dewayne Johnson,Black,Male,1999,"April 12, 1999"
9,Next day,1971,"November 1, 1989","""There is no way no words can express how sorr...",Unknown,Unknown,Unknown,Unknown,Unknown,http://deathpenaltyusa.org/usa/female/riggs-ch...,Arkansas,Pulaski,2-May-00,http://deathpenaltyusa.org/usa/female/riggs-ch...,Lethal Injection,Christina Marie Riggs,White,Female,1999,"February 16, 1999"


In [114]:
df_ark.to_csv('final_arkansas.csv', index=False)

# California

In [116]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/california.htm')

In [117]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "California"
    rows_df.append(row_df)
#print(rows_df)
df_california = pd.DataFrame(rows_df)
df_california

,State,county,date_execution,link_to_profile,method,name
0,California,Glenn,"January 17, 2006",http://murderpedia.org/male.A/a1/allen-clarenc...,Lethal Injection,Clarence Ray Allen
1,California,Los Angeles,"December 13, 2005",http://murderpedia.org/male.W/w1/williams-stan...,Lethal Injection,Stanley Williams
2,California,San Mateo,"January 19, 2005",http://murderpedia.org/male.B/b1/beardslee-don...,Lethal Injection,Donald Jay Beardslee
3,California,San Bernardino,"January 29, 2002",http://murderpedia.org/male.A/a1/anderson-step...,Lethal Injection,Stephen Wayne Anderson
4,California,San Francisco,"March 27, 2001",http://murderpedia.org/male.M/m1/massie-robert...,Lethal Injection,Robert Lee Massie
5,California,Shasta,"March 15, 2000",http://murderpedia.org/male.R/r1/rich-darrell-...,Lethal Injection,Darrel Keith Rich
6,California,Sacramento,"May 4, 1999",http://murderpedia.org/male.B/b1/babbit-manuel...,Lethal Injection,Manuel Pina Babbit
7,California,Orange,"February 9, 1999",http://murderpedia.org/male.S/s1/siripongs-jat...,Lethal Injection,Jaturun Siripongs
8,California,Orange,"July 14, 1998",http://murderpedia.org/male.T/t1/thompson-thom...,Lethal Injection,Thomas Martin Thompson
9,California,Merced,"May 31, 1996",http://murderpedia.org/male.W/w1/williams-keit...,Lethal Injection,Keith Daniel Williams


In [118]:
df_all_california = df_california.apply(scrape_each_execution, axis=1)
df_all_california

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND
1,NOT FOUND,"December 29, 1953",February/March 1979,None.,Executed by lethal injection in California on ...,NOT FOUND,Robberies - Co-founder of the violent Crips st...,NOT FOUND,4
2,"April 26, 1981","May 13, 1943",1969 / 1981,None.,Executed by lethal injection in California on ...,NOT FOUND,Drugs,NOT FOUND,3
3,"May 26, 1980","July 8, 1953",1970's - 1980,None.,Executed by lethal injection in California on ...,NOT FOUND,Robbery - Contract killings,NOT FOUND,8
4,"January 4, 1979","December 24, 1941",1965 / 1979,NOT FOUND,Executed by lethal injection in California on ...,NOT FOUND,Robberies,NOT FOUND,2
5,"August 23, 1978","February 14, 1955",June-August 1978,NOT FOUND,Executed by lethal injection in California on ...,NOT FOUND,Rape,NOT FOUND,4
6,NOT FOUND,1949,"December 19, 1980",NOT FOUND,Executed by lethal injection in California on ...,NOT FOUND,Rape - Robbery,NOT FOUND,1
7,2 days after,1955,"December 15, 1981",NOT FOUND,Executed by lethal injection in California on ...,NOT FOUND,Robbery,NOT FOUND,2
8,NOT FOUND,"March 20, 1955","September 12, 1981",NOT FOUND,Executed by lethal injection in California on ...,NOT FOUND,Rape,NOT FOUND,1
9,NOT FOUND,"June 6, 1947",October 8. 1978,NOT FOUND,Executed by lethal injection in California on ...,NOT FOUND,Robbery - Kidnapping - Rape,NOT FOUND,3


In [119]:
df_all_california['url'] = df_california.link_to_profile 
df_all_california = df_all_california.merge(df_california, right_on='link_to_profile', left_on = 'url')
df_all_california

,date_arrest,date_of_birth,date_offense,final_words,method_x,mitigating_circumstances,offense,race,victim_num,url,State,county,date_execution,link_to_profile,method_y,name
0,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,http://murderpedia.org/male.A/a1/allen-clarenc...,California,Glenn,"January 17, 2006",http://murderpedia.org/male.A/a1/allen-clarenc...,Lethal Injection,Clarence Ray Allen
1,NOT FOUND,"December 29, 1953",February/March 1979,None.,Executed by lethal injection in California on ...,NOT FOUND,Robberies - Co-founder of the violent Crips st...,NOT FOUND,4,http://murderpedia.org/male.W/w1/williams-stan...,California,Los Angeles,"December 13, 2005",http://murderpedia.org/male.W/w1/williams-stan...,Lethal Injection,Stanley Williams
2,"April 26, 1981","May 13, 1943",1969 / 1981,None.,Executed by lethal injection in California on ...,NOT FOUND,Drugs,NOT FOUND,3,http://murderpedia.org/male.B/b1/beardslee-don...,California,San Mateo,"January 19, 2005",http://murderpedia.org/male.B/b1/beardslee-don...,Lethal Injection,Donald Jay Beardslee
3,"May 26, 1980","July 8, 1953",1970's - 1980,None.,Executed by lethal injection in California on ...,NOT FOUND,Robbery - Contract killings,NOT FOUND,8,http://murderpedia.org/male.A/a1/anderson-step...,California,San Bernardino,"January 29, 2002",http://murderpedia.org/male.A/a1/anderson-step...,Lethal Injection,Stephen Wayne Anderson
4,"January 4, 1979","December 24, 1941",1965 / 1979,NOT FOUND,Executed by lethal injection in California on ...,NOT FOUND,Robberies,NOT FOUND,2,http://murderpedia.org/male.M/m1/massie-robert...,California,San Francisco,"March 27, 2001",http://murderpedia.org/male.M/m1/massie-robert...,Lethal Injection,Robert Lee Massie
5,"August 23, 1978","February 14, 1955",June-August 1978,NOT FOUND,Executed by lethal injection in California on ...,NOT FOUND,Rape,NOT FOUND,4,http://murderpedia.org/male.R/r1/rich-darrell-...,California,Shasta,"March 15, 2000",http://murderpedia.org/male.R/r1/rich-darrell-...,Lethal Injection,Darrel Keith Rich
6,NOT FOUND,1949,"December 19, 1980",NOT FOUND,Executed by lethal injection in California on ...,NOT FOUND,Rape - Robbery,NOT FOUND,1,http://murderpedia.org/male.B/b1/babbit-manuel...,California,Sacramento,"May 4, 1999",http://murderpedia.org/male.B/b1/babbit-manuel...,Lethal Injection,Manuel Pina Babbit
7,2 days after,1955,"December 15, 1981",NOT FOUND,Executed by lethal injection in California on ...,NOT FOUND,Robbery,NOT FOUND,2,http://murderpedia.org/male.S/s1/siripongs-jat...,California,Orange,"February 9, 1999",http://murderpedia.org/male.S/s1/siripongs-jat...,Lethal Injection,Jaturun Siripongs
8,NOT FOUND,"March 20, 1955","September 12, 1981",NOT FOUND,Executed by lethal injection in California on ...,NOT FOUND,Rape,NOT FOUND,1,http://murderpedia.org/male.T/t1/thompson-thom...,California,Orange,"July 14, 1998",http://murderpedia.org/male.T/t1/thompson-thom...,Lethal Injection,Thomas Martin Thompson
9,NOT FOUND,"June 6, 1947",October 8. 1978,NOT FOUND,Executed by lethal injection in California on ...,NOT FOUND,Robbery - Kidnapping - Rape,NOT FOUND,3,http://murderpedia.org/male.W/w1/williams-keit...,California,Merced,"May 31, 1996",http://murderpedia.org/male.W/w1/williams-keit...,Lethal Injection,Keith Daniel Williams


In [121]:
df_all_california['year'] = df_all_california['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_california.to_csv('final_california.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


# Colorado 

In [123]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/colorado.htm')

In [124]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Arkansas"
    rows_df.append(row_df)
#print(rows_df)
df_colorado = pd.DataFrame(rows_df)
df_colorado

,State,county,date_execution,link_to_profile,method,name
0,Arkansas,Arapahoe,"October 13, 1997",http://murderpedia.org/male.D/d1/davis-gary-le...,Lethal Injection,Gary Lee Davis


In [125]:
df_all_colorado = df_colorado.apply(scrape_each_execution, axis=1)
df_all_colorado

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,NOT FOUND,1944,"July 21, 1986",NOT FOUND,Executed by lethal injection in Colorado on Oc...,NOT FOUND,Kidnapping - Rape,NOT FOUND,1


In [126]:
df_all_colorado['url'] = df_colorado.link_to_profile 
df_all_colorado = df_all_colorado.merge(df_colorado, right_on='link_to_profile', left_on = 'url')
df_all_colorado['year'] = df_all_colorado['date_execution'].str.extract(r"(\d\d\d\d)")


,date_arrest,date_of_birth,date_offense,final_words,method_x,mitigating_circumstances,offense,race,victim_num,url,State,county,date_execution,link_to_profile,method_y,name
0,NOT FOUND,1944,"July 21, 1986",NOT FOUND,Executed by lethal injection in Colorado on Oc...,NOT FOUND,Kidnapping - Rape,NOT FOUND,1,http://murderpedia.org/male.D/d1/davis-gary-le...,Arkansas,Arapahoe,"October 13, 1997",http://murderpedia.org/male.D/d1/davis-gary-le...,Lethal Injection,Gary Lee Davis


In [127]:
df_all_colorado.to_csv('colorado-2.csv', index=False)

# Connecticut

In [128]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/connecticut.htm')

In [129]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Arkansas"
    rows_df.append(row_df)
#print(rows_df)
df_connecticut = pd.DataFrame(rows_df)
df_connecticut

,State,county,date_execution,link_to_profile,method,name
0,Arkansas,New London,"May 13, 2005",http://murderpedia.org/male.R/r1/ross-michael.htm,Lethal Injection,Michael Bruce Ross


In [130]:
df_all_connecticut = df_connecticut.apply(scrape_each_execution, axis=1)
df_all_connecticut

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,"June 28, 1984","July 26, 1959",1981 - 1984,None.,Executed by lethal injection in Connecticut on...,NOT FOUND,Rape,NOT FOUND,8


In [132]:
df_all_connecticut['url'] = df_connecticut.link_to_profile 
df_all_connecticut = df_all_connecticut.merge(df_connecticut, right_on='link_to_profile', left_on = 'url')
df_all_connecticut['year'] = df_all_connecticut['date_execution'].str.extract(r"(\d\d\d\d)")

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [133]:
df_all_connecticut.to_csv('connecticut-2.csv', index=False)

# Delaware

In [134]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/delaware.htm')

In [135]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Delaware"
    rows_df.append(row_df)
#print(rows_df)
df_delaware = pd.DataFrame(rows_df)
df_delaware

,State,county,date_execution,link_to_profile,method,name
0,Delaware,New Castle,"April 20, 2012",http://murderpedia.org/male.J/j1/johnson-shann...,Lethal Injection,Shannon M. Johnson
1,Delaware,New Castle,"July 29, 2011",http://murderpedia.org/male.J/j1/jackson-rober...,Lethal Injection,Robert W. Jackson III
2,Delaware,New Castle,"November 4, 2005",http://murderpedia.org/male.S/s1/steckel-brian...,Lethal Injection,Brian David Steckel
3,Delaware,New Castle,"May 25, 2001",http://murderpedia.org/male.H/h1/hameen-abdull...,Lethal Injection,Abdullah Tanzil Hameen
4,Delaware,Kent,"April 26, 2001",http://murderpedia.org/male.D/d1/dawson-david.htm,Lethal Injection,David F. Dawson
5,Delaware,New Castle,"November 17, 2000",http://murderpedia.org/male.W/w1/weeks-dwayne.htm,Lethal Injection,Dwayne L. Weeks
6,Delaware,Kent,"September 24, 1999",http://murderpedia.org/male.S/s1/sullivan-will...,Lethal Injection,Willie G. Sullivan
7,Delaware,Kent,"April 23, 1999",http://murderpedia.org/male.L/l1/lawrie-david.htm,Lethal Injection,David J. Laurie
8,Delaware,Cochise,"April 19, 1996",http://murderpedia.org/male.C/c1/clark-james-b...,Lethal Injection,James B. Clark Jr.
9,Delaware,Kent,"January 30, 1996",http://murderpedia.org/male.F/f1/flamer-willia...,Lethal Injection,William Henry Flamer


In [139]:
df_all_delaware = df_delaware.apply(scrape_each_execution_arkansas, axis=1)
df_all_delaware

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,"November 15, 2006","November 18, 1983","September 24, 2006","""Loyalty is important. Without loyalty you hav...",Executed by lethal injection in Delaware on Ap...,NOT FOUND,The victim had been dating Johnson's ex-girlfr...,Black,1
1,"April 19, 1992","May 24, 1963","April 3, 1992","""Are the Girardis in there? Christopher and Cl...","Sentenced to death on April 28, 1993. Executed...",NOT FOUND,Robbery,White,1
2,NOT FOUND,"December 14, 1968","September 2, 1994","He repeatedly apologized for his crimes, profe...",Executed by lethal injection in Delaware on No...,NOT FOUND,Rape - Arson,White Gender: Male,1 +
3,"October 23, 1991","October 7, 1963",1980 / 1991,NOT FOUND,Executed by lethal injection in Delaware on Ma...,NOT FOUND,Juvenile (17) - Drugs,NOT FOUND,2
4,Next day,"April 1, 1955","December 1, 1986",NOT FOUND,Executed by lethal injection in Delaware on Ap...,NOT FOUND,Fugitive - Robbery,NOT FOUND,1
5,Same day,"August 19, 1963","April 10, 1992",NOT FOUND,Executed by lethal injection in Delaware on No...,NOT FOUND,Parricide - Revenge,NOT FOUND,2
6,"January 3, 1992","July 17, 1972","December 27, 1991",NOT FOUND,Executed by lethal injection in Delaware on Se...,NOT FOUND,Robbery,NOT FOUND,1
7,Same day (surrenders),"June 29, 1961","August 6, 1992",NOT FOUND,Executed by lethal injection in Delaware on Ap...,NOT FOUND,Parricide,NOT FOUND,4
8,Several days later,"February 8, 1957","May 22, 1994",NOT FOUND,"Executed by lethal injection on April 19, 1996",NOT FOUND,Parricide - To collect insurance money,NOT FOUND,2
9,Next day,1954,"February 7, 1979",NOT FOUND,Executed by lethal injection in Delaware on Ja...,NOT FOUND,Robbery,NOT FOUND,2


In [140]:
df_all_delaware['url'] = df_delaware.link_to_profile 
df_all_delaware = df_all_delaware.merge(df_delaware, right_on='link_to_profile', left_on = 'url')
df_all_delaware['year'] = df_all_delaware['date_execution'].str.extract(r"(\d\d\d\d)")

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [141]:
df_all_delaware.to_csv('delaware-2.csv', index=False)

# Florida

In [142]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/florida.htm')

In [143]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Florida"
    rows_df.append(row_df)
#print(rows_df)
df_florida = pd.DataFrame(rows_df)
df_florida

,State,county,date_execution,link_to_profile,method,name
0,Florida,Polk,"July 10, 2014",http://murderpedia.org/male.D/d/davis-eddie-wa...,Lethal Injection,Eddie Wayne Davis
1,Florida,Pasco,"June 18, 2014",http://murderpedia.org/male.H/h/henry-john-rut...,Lethal Injection,John Ruthell Henry
2,Florida,Lake,"April 23, 2014",http://murderpedia.org/male.H/h/hendrix-robert...,Lethal Injection,Robert Eugene Hendrix
3,Florida,Broward,"March 20, 2014",http://murderpedia.org/male.H/h/henry-robert-l...,Lethal Injection,Robert L. Henry
4,Florida,Jefferson,"February 26, 2014",http://murderpedia.org/male.H/h/howell-paul.htm,Lethal Injection,Paul Augustus Howell
5,Florida,Orange,"February 12, 2014",http://murderpedia.org/male.C/c/chavez-juan-ca...,Lethal Injection,Juan Carlos Chavez
6,Florida,Bradford,"January 7, 2014",http://murderpedia.org/male.M/m1/muhammad-aska...,Lethal Injection,Askari Abdullah Muhammad
7,Florida,Orange,"November 12, 2013",http://murderpedia.org/male.K/k/kimbrough-dari...,Lethal Injection,Darius Mark Kimbrough
8,Florida,Citrus,"October 15, 2013",http://murderpedia.org/male.H/h/happ-william.htm,Lethal Injection,William Frederick Happ
9,Florida,Columbia,"October 1, 2013",http://murderpedia.org/male.G/g/gore-marshall-...,Lethal Injection,Marshall Lee Gore


In [146]:
df_all_florida = df_florida.apply(scrape_each_execution_arkansas, axis=1)
df_all_florida

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,NOT FOUND,"September 13, 1968","March 3, 1994",Declined.,"Sentenced to death on June 30, 1995. Executed ...",NOT FOUND,Child molester,White,1
1,NOT FOUND,"January 16, 1951",1975 / 1985,NOT FOUND,Sentenced to 15 years in prison in 1976. Parol...,NOT FOUND,Parricide,NOT FOUND,3
2,NOT FOUND,"October 21, 1966","August 27, 1990",NOT FOUND,"Sentenced to death on November 4, 1991. Execut...",NOT FOUND,To prevent Scott from testifying against him,NOT FOUND,2
3,Next day,"May 1, 1958","November 2, 1987",NOT FOUND,"Sentenced to death on November 9, 1988. Execut...",NOT FOUND,Robbery,NOT FOUND,2
4,Same day,"June 25, 1965","February 1, 1992","Asked if he had any words, he apologized to Fu...","Sentenced to death on January 10, 1995. Execut...",NOT FOUND,Bomber - Drug ring,BLACK,1
5,"December 6, 1995","March 16, 1967","September 11, 1995",None.,"Sentenced to death on November 23, 1998. Execu...",NOT FOUND,Kidnapping - Rape,NOT FOUND,1
6,"December 24, 1974","February 4, 1951",1974 / 1980,NOT FOUND,"Sentenced to death in Florida on April 21, 197...",NOT FOUND,Kidnapping - Robberies,NOT FOUND,4
7,5 months later,"December 4, 1972","October 3, 1991",NOT FOUND,"Sentenced to death on December 9, 1994. Execut...",NOT FOUND,Rape - Robbery,NOT FOUND,1
8,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND
9,"March 17, 1988","August 17, 1963","January 31/March 11, 1988",NOT FOUND,"Sentenced to death on April 3, 1990. Executed ...",NOT FOUND,Serial rapist - Robberies,NOT FOUND,2


In [147]:
df_all_florida['url'] = df_florida.link_to_profile 
df_all_florida = df_all_florida.merge(df_florida, right_on='link_to_profile', left_on = 'url')
df_all_florida['year'] = df_all_florida['date_execution'].str.extract(r"(\d\d\d\d)")

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [148]:
df_all_florida.to_csv('florida-2.csv', index=False)

# Georgia

In [149]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/georgia.htm')

In [151]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Georgia"
    rows_df.append(row_df)
#print(rows_df)
df_georgia = pd.DataFrame(rows_df)
df_georgia

,State,county,date_execution,link_to_profile,method,name
0,Georgia,Cobb,"June 17, 2014",http://murderpedia.org/male.W/w/wellons-marcus...,Lethal Injection,Marcus A. Wellons
1,Georgia,Monroe,"February 21, 2013",http://murderpedia.org/male.C/c1/cook-andrew-a...,Lethal Injection,Andrew Allen Cook
2,Georgia,Chatham,"September 21, 2011",http://murderpedia.org/male.D/d1/davis-troy-an...,Lethal Injection,Troy Anthony Davis
3,Georgia,Cobb,"July 21, 2011",http://murderpedia.org/male.D/d1/deyoung-andre...,Lethal Injection,Andrew Grant DeYoung
4,Georgia,Chatham,"June 23, 2011",http://murderpedia.org/male.B/b1/blankenship-r...,Lethal Injection,Roy Willard Blankenship
5,Georgia,Fulton,"January 25, 2011",http://murderpedia.org/male.H/h1/hammond-emman...,Lethal Injection,Emmanuel Fitzgerald Hammond
6,Georgia,Jones,"September 27, 2010",http://murderpedia.org/male.R/r1/rhode-brandon...,Lethal Injection,Brandon Joseph Rhode
7,Georgia,Newton,"June 9, 2010",http://murderpedia.org/male.F/f1/ford-melbert-...,Lethal Injection,Melbert Ray Ford Jr.
8,Georgia,Richmond,"October 20, 2009",http://murderpedia.org/male.M/m1/mcclain-mark-...,Lethal Injection,Mark Howard McClain
9,Georgia,Oconee,"April 29, 2009",http://murderpedia.org/male.M/m1/mize-william-...,Lethal Injection,William Mark Mize


In [152]:
df_all_georgia = df_georgia.apply(scrape_each_execution_arkansas, axis=1)
df_all_georgia

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,Same day,1955,"August 31, 1989",NOT FOUND,"Sentenced to death on June 9, 1993.Executed by...",NOT FOUND,Rape - Revenge,NOT FOUND,1
1,"December 5, 1996",July 1974,"January 2, 1995","“I’m sorry,” Cook said as he was strapped to a...","Sentenced to death on March 19, 1997. Executed...",NOT FOUND,The murders were completely random,NOT FOUND,2
2,NOT FOUND,"October 6, 1968","August 18, 1989","""I'd like to address the MacPhail family. Let ...","Sentenced to death on August 30, 1991. Execute...",NOT FOUND,To avoid arrest,NOT FOUND,1
3,Same day,"May 12, 1974","June 14, 1993","""I'm sorry for everyone I've hurt.""","Sentenced to death on October 13, 1995. Execut...",NOT FOUND,Parricide - To inherit,NOT FOUND,3
4,"March 17, 1978",1955,"March 2, 1978","""I hope to see you again.""",Executed by lethal injection in Georgia on Jun...,NOT FOUND,Rape,NOT FOUND,1
5,August 1989,"June 6, 1965","July 12, 1988",None,Executed by lethal injection in Georgia on Jan...,NOT FOUND,Serial rapist - Kidnapping - Robbery,NOT FOUND,1
6,NOT FOUND,"July 16, 1979","April 23, 1998",None.,Executed by lethal injection in Georgia on Sep...,NOT FOUND,Robbery,NOT FOUND,3
7,Next day,1960,"March 6, 1986","""I wish to thank all my family and my friends ...",Executed by lethal injection in Georgia on Jun...,NOT FOUND,Revenge - Robbery,NOT FOUND,2
8,Next day,"June 1, 1967","November 20, 1994",When prison Warden Steve Upton asked him wheth...,Executed by lethal injection in Georgia on Oct...,NOT FOUND,Robbery,NOT FOUND,1
9,NOT FOUND,"July 30, 1956","October 16, 1994","Mize, in his final statement to a room of abou...",Executed by lethal injection in Georgia on Apr...,NOT FOUND,Leader white supremacist group National Vastil...,NOT FOUND,1


In [154]:
df_all_georgia['url'] = df_georgia.link_to_profile 
df_all_georgia = df_all_georgia.merge(df_georgia, right_on='link_to_profile', left_on = 'url')
df_all_georgia['year'] = df_all_georgia['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_georgia.to_csv('georgia-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Idaho

In [156]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/idaho.htm')

In [157]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Idaho"
    rows_df.append(row_df)
#print(rows_df)
df_idaho = pd.DataFrame(rows_df)
df_idaho

,State,county,date_execution,link_to_profile,method,name
0,Idaho,Bingham,"June 12, 2012",http://murderpedia.org/male.L/l/leavitt-richar...,Lethal Injection,Richard Albert Leavitt
1,Idaho,Bonneville,"November 18, 2011",http://murderpedia.org/male.R/r1/rhoades-paul.htm,Lethal Injection,Paul Ezra Rhoades
2,Idaho,Ada,"January 6, 1994",http://murderpedia.org/male.W/w1/wells-keith.htm,Lethal Injection,Keith Eugene Wells


In [158]:
df_all_idaho = df_idaho.apply(scrape_each_execution_arkansas, axis=1)
df_all_idaho

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,NOT FOUND,"November 12, 1958","July 18, 1984",None.,"Sentenced to death on December 19, 1985. Execu...",NOT FOUND,Mutilation (remove her sexual organs),NOT FOUND,1
1,"March 25, 1987","January 18, 1957",1985 - 1987,"""To Bert Michelbacher, I am sorry for the part...",Sentenced to death in Idaho in March 1988. Exe...,NOT FOUND,Rape - Robberies,NOT FOUND,6
2,NOT FOUND,"May 11, 1962","December 20, 1990",NOT FOUND,Executed by lethal injection in Idaho on Janua...,NOT FOUND,"No reason other than that ""it was time for the...",NOT FOUND,2


In [159]:
df_all_idaho['url'] = df_idaho.link_to_profile 
df_all_idaho = df_all_idaho.merge(df_idaho, right_on='link_to_profile', left_on = 'url')
df_all_idaho['year'] = df_all_idaho['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_idaho.to_csv('idaho-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Illinois

In [160]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/illinois.htm')

In [161]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Illinois"
    rows_df.append(row_df)
#print(rows_df)
df_illinois = pd.DataFrame(rows_df)
df_illinois

,State,county,date_execution,link_to_profile,method,name
0,Illinois,Cook,"March 17, 1999",http://murderpedia.org/male.K/k1/kokoraleis-an...,Lethal Injection,Andrew Kokoraleis
1,Illinois,Madison,"January 21, 1998",http://murderpedia.org/male.H/h1/hampton-lloyd...,Lethal Injection,Lloyd Wayne Hampton
2,Illinois,Cook,"November 19, 1997",http://murderpedia.org/male.E/e1/eddmonds-durl...,Lethal Injection,Durlyn Eddmonds
3,Illinois,Cook,"November 19, 1997",http://murderpedia.org/male.S/s1/stewart-walte...,Lethal Injection,Walter Stewart
4,Illinois,Winnebago,"September 18, 1996",http://murderpedia.org/male.S/s1/stewart-raymo...,Lethal Injection,Raymond Lee Steward
5,Illinois,Cook,"November 22, 1995",http://murderpedia.org/male.D/d1/del-vecchio-g...,Lethal Injection,George W. Del Vecchio
6,Illinois,McHenry,"September 20, 1995",http://murderpedia.org/male.A/a1/albanese-char...,Lethal Injection,Charles Albanese
7,Illinois,St. Clair,"May 17, 1995",http://murderpedia.org/male.D/d1/davis-girvies...,Lethal Injection,Girvies L. Davis
8,Illinois,DuPage,"April 6, 1995",http://murderpedia.org/male.F/f1/free-james.htm,Lethal Injection,James P. Free Jr.
9,Illinois,Cook,"March 22, 1995",http://murderpedia.org/male.W/w1/williams-hern...,Lethal Injection,Hernando Williams


In [162]:
df_all_illinois = df_illinois.apply(scrape_each_execution_arkansas, axis=1)
df_all_illinois

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,October 1982,1961,1981 - 1982,NOT FOUND,Executed by lethal injection in Illinois on Ma...,NOT FOUND,Kidnapping - Rape - Torture - Mutilation - Can...,NOT FOUND,18 +
1,Next day,1954,"February 8, 1990",NOT FOUND,Executed by lethal injection in Illinois on Ja...,NOT FOUND,Robbery - Torture,NOT FOUND,1
2,NOT FOUND,1952,"October 27, 1977",NOT FOUND,Executed by lethal injection in Illinois on No...,NOT FOUND,Rape,NOT FOUND,1
3,Same day,1956,"February 10, 1980",NOT FOUND,Executed by lethal injection in Illinois on No...,NOT FOUND,Jewelry store robbery,NOT FOUND,2
4,NOT FOUND,1952,January-February 1981,NOT FOUND,Executed by lethal injection in Illinois on Se...,NOT FOUND,Robberies - Revenge,NOT FOUND,6
5,"December 22, 1977",1948,1965 / 1977,NOT FOUND,Executed by lethal injection in Illinois on No...,NOT FOUND,Juvenile (16) - Rape,NOT FOUND,2
6,NOT FOUND,1946,1980 - 1981,NOT FOUND,Executed by lethal injection in Illinois on Se...,NOT FOUND,Poisoner - Parricide,NOT FOUND,3
7,NOT FOUND,"January 20, 1958",1978 - 1980,NOT FOUND,Executed by lethal injection in Illinois on Ma...,NOT FOUND,Robberies,NOT FOUND,4
8,NOT FOUND,1958,"April 24, 1978",NOT FOUND,Executed by lethal injection in Illinois on Ap...,NOT FOUND,Rape,NOT FOUND,1
9,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND


In [163]:
df_all_illinois['url'] = df_illinois.link_to_profile 
df_all_illinois = df_all_illinois.merge(df_illinois, right_on='link_to_profile', left_on = 'url')
df_all_illinois['year'] = df_all_illinois['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_illinois.to_csv('illinois-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Kentucky 

In [164]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/kentucky.htm')

In [165]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Kentucky"
    rows_df.append(row_df)
#print(rows_df)
df_kentucky = pd.DataFrame(rows_df)
df_kentucky

,State,county,date_execution,link_to_profile,method,name
0,Kentucky,Gallatin,"November 21, 2008",http://murderpedia.org/male.C/c1/chapman-marco...,Lethal Injection,Marco Allen Chapman
1,Kentucky,Jefferson,"May 25, 1999",http://murderpedia.org/male.H/h1/harper-edward...,Lethal Injection,Edward L. Harper
2,Kentucky,Madison,"July 1, 1997",http://murderpedia.org/male.M/m1/mcqueen-harol...,Electrocution,Harold I. McQueen Jr.


In [166]:
df_all_kentucky = df_kentucky.apply(scrape_each_execution_arkansas, axis=1)
df_all_kentucky

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,Same day,"September 4, 1971","August 23, 2002","""I don't know why I did the thing that I did, ...",Executed by lethal injection in Kentucky on No...,NOT FOUND,Rape - Robbery,NOT FOUND,2
1,NOT FOUND,1948,"February 19, 1982",NOT FOUND,Executed by lethal injection in Kentucky on Ma...,NOT FOUND,Parricide - Attempt to collect insurance money,NOT FOUND,2
2,NOT FOUND,"July 25, 1952","January 17, 1981",NOT FOUND,Executed by electrocution in Kentucky on July ...,NOT FOUND,Robbery,NOT FOUND,1


In [167]:
df_all_kentucky['url'] = df_kentucky.link_to_profile 
df_all_kentucky = df_all_kentucky.merge(df_kentucky, right_on='link_to_profile', left_on = 'url')
df_all_kentucky['year'] = df_all_kentucky['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_kentucky.to_csv('kentucky-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Louisiana

In [168]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/louisiana.htm')

In [169]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Louisiana"
    rows_df.append(row_df)
#print(rows_df)
df_louisiana = pd.DataFrame(rows_df)
df_louisiana

,State,county,date_execution,link_to_profile,method,name
0,Louisiana,E. Baton Rouge,"January 7, 2010",http://murderpedia.org/male.B/b1/bordelon-gera...,Lethal Injection,Gerald J. Bordelon
1,Louisiana,Calcasieu,"May 10, 2002",http://murderpedia.org/male.M/m1/martin-leslie...,Lethal Injection,Leslie Dale Martinez
2,Louisiana,E. Baton Rouge,"June 6, 2000",http://murderpedia.org/male.T/t1/taylor-feltus...,Lethal Injection,Feltus Taylor Jr.
3,Louisiana,Sabine,"January 8, 1999",http://murderpedia.org/male.W/w1/williams-dobi...,Lethal Injection,Dobie Gillis Williams
4,Louisiana,Orleans,"April 24, 1997",http://murderpedia.org/male.B/b1/brown-john-as...,Lethal Injection,John Ashley Brown Jr.
5,Louisiana,Orleans,"March 1, 1996",http://murderpedia.org/male.J/j1/james-antonio...,Lethal Injection,Antonio G. James
6,Louisiana,Orleans,"March 16, 1995",http://murderpedia.org/male.W/w1/ward-thomas-l...,Lethal Injection,Thomas Lee Ward
7,Louisiana,Jefferson,"March 5, 1993",http://murderpedia.org/male.S/s1/sawyer-robert...,Lethal Injection,Robert Wayne Sawyer
8,Louisiana,Baton Rouge,"July 22, 1991",http://murderpedia.org/male.J/j1/jones-andrew-...,Electrocution,Andrew Lee Jones
9,Louisiana,Lafayette,"May 18, 1990",http://murderpedia.org/male.P/p1/prejean-dalto...,Electrocution,Dalton Prejean


In [173]:
df_all_louisiana = df_louisiana.apply(scrape_each_execution_arkansas, axis=1)
df_all_louisiana

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,NOT FOUND,"February 19, 1962","November 15, 2002","Just before his execution, Bordelon apologized...",Executed by lethal injection in Louisiana on J...,NOT FOUND,Rape,NOT FOUND,1
1,July 1991,"April 24, 1967","June 20, 1991",Martin declines to make a final statement befo...,Executed by lethal injection in Louisiana on M...,NOT FOUND,Rape,NOT FOUND,1
2,Same day,1962,"March 27, 1991",NOT FOUND,Executed by lethal injection in Louisiana on J...,NOT FOUND,Robbery,NOT FOUND,1
3,Same day,1961,"July 8, 1984",NOT FOUND,Executed by lethal injection in Louisiana on J...,NOT FOUND,Robbery,NOT FOUND,1
4,Same day,1961,"September 7, 1984",NOT FOUND,Executed by lethal injection in Louisiana on A...,NOT FOUND,Robbery,NOT FOUND,1
5,"January 26, 1979",1954,"January 1/23, 1979",NOT FOUND,Executed by lethal injection in Louisiana on M...,NOT FOUND,Robberies,NOT FOUND,2
6,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND
7,Same day,1951,"September 28, 1979",NOT FOUND,Executed by lethal injection in Louisiana on M...,NOT FOUND,Rape - Mutilation,NOT FOUND,1
8,2 days after,1955,"February 17, 1984",NOT FOUND,Executed by electrocution in Louisiana on July...,NOT FOUND,Rape,NOT FOUND,1
9,"July 2, 1977","December 10, 1959","June 1974 / July 2, 1977",NOT FOUND,Executed by electrocution in Louisiana on May ...,NOT FOUND,Juvenile (17) - Robbery - To avoid arrest,NOT FOUND,2


In [174]:
df_all_louisiana['url'] = df_louisiana.link_to_profile 
df_all_louisiana = df_all_louisiana.merge(df_louisiana, right_on='link_to_profile', left_on = 'url')
df_all_louisiana['year'] = df_louisiana['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_louisiana.to_csv('louisiana-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Maryland

In [175]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/maryland.htm')

In [176]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Maryland"
    rows_df.append(row_df)
#print(rows_df)
df_maryland = pd.DataFrame(rows_df)
df_maryland

,State,county,date_execution,link_to_profile,method,name
0,Maryland,Baltimore,"December 5, 2005",http://murderpedia.org/male.B/b1/baker-wesley-...,Lethal Injection,Wesley Eugene Baker
1,Maryland,Baltimore,"June 17, 2004",http://murderpedia.org/male.O/o1/oken-steven.htm,Lethal Injection,Steven Howard Oken
2,Maryland,Baltimore,"November 16, 1998",http://murderpedia.org/male.G/g1/gilliam-tyron...,Lethal Injection,Tyrone X. Gilliam
3,Maryland,Baltimore City,"July 2, 1997",http://murderpedia.org/male.H/h1/hunt-flint-gr...,Lethal Injection,Flint Gregory Hunt
4,Maryland,Baltimore,"May 16, 1994",http://murderpedia.org/male.T/t1/thanos-john-f...,Lethal Injection,John Frederick Thanos


In [177]:
df_all_maryland = df_maryland.apply(scrape_each_execution_arkansas, axis=1)
df_all_maryland

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,Same day,"March 26, 1958","June 6, 1991",None.,Executed by lethal injection in Maryland on De...,NOT FOUND,Robbery,NOT FOUND,1
1,"November 17, 1987","January 22, 1962","November 1-16, 1987",None.,Executed by lethal injection in Maryland on Ju...,NOT FOUND,Rape - Sadism,NOT FOUND,3
2,NOT FOUND,1956,"December 2, 1988",NOT FOUND,Executed by lethal injection in Maryland on No...,NOT FOUND,Abduction - Robbery - Drugs,NOT FOUND,1
3,5 days after,1959,"November 18, 1985",NOT FOUND,Executed by lethal injection in Maryland on Ju...,NOT FOUND,To avoid arrest,NOT FOUND,1
4,NOT FOUND,1949,August-September 1990,NOT FOUND,Executed by lethal injection in Maryland on Ma...,NOT FOUND,Robberies,NOT FOUND,3


In [178]:
df_all_maryland['url'] = df_maryland.link_to_profile 
df_all_maryland = df_all_maryland.merge(df_maryland, right_on='link_to_profile', left_on = 'url')
df_all_maryland['year'] = df_maryland['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_maryland.to_csv('maryland-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Mississippi

In [179]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/mississippi.htm')

In [180]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Mississippi"
    rows_df.append(row_df)
#print(rows_df)
df_mississippi = pd.DataFrame(rows_df)
df_mississippi

,State,county,date_execution,link_to_profile,method,name
0,Mississippi,Jackson,"June 20, 2012",http://murderpedia.org/male.S/s1/simmons-gary-...,Lethal Injection,Gary Carl Simmons Jr.
1,Mississippi,Tate,"June 12, 2012",http://murderpedia.org/male.B/b1/brawner-jan-m...,Lethal Injection,Jan Michael Brawner
2,Mississippi,Leflore,"June 5, 2012",http://murderpedia.org/male.J/j/jackson-henry-...,Lethal Injection,Henry Curtis Jackson Jr.
3,Mississippi,Harrison,"March 22, 2012",http://murderpedia.org/male.M/m/mitchell-willi...,Lethal Injection,William Gerald Mitchell
4,Mississippi,Forrest,"March 20, 2012",http://murderpedia.org/male.P/p/puckett-larry-...,Lethal Injection,Larry Matthew Puckett
5,Mississippi,Carroll,"February 8, 2012",http://murderpedia.org/male.T/t1/turner-edwin-...,Lethal Injection,Edwin Hart Turner
6,Mississippi,Newton,"May 17, 2011",http://murderpedia.org/male.G/g1/gray-rodney.htm,Lethal Injection,Rodney Gray
7,Mississippi,Marion,"May 10, 2011",http://murderpedia.org/male.S/s1/stevens-benny...,Lethal Injection,Benny Joe Stevens
8,Mississippi,Lee,"July 21, 2010",http://murderpedia.org/male.B/b1/burns-joseph-...,Lethal Injection,Joseph Daniel Burns
9,Mississippi,Harrison,"May 20, 2010",http://murderpedia.org/male.H/h1/holland-geral...,Lethal Injection,Gerald James Holland


In [181]:
df_all_maryland = df_mississippi.apply(scrape_each_execution_arkansas, axis=1)
df_all_maryland

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,Next day (surrenders),"November 18, 1962","August 11, 1996",“I’ve been blessed to be loved by some good pe...,"Sentenced to death on August 29, 1997. Execute...",NOT FOUND,Robbery - Rape - Drugs - Dismemberment,WHITE,1
1,Same day,"June 10, 1977","April 25, 2001","In his final statement, Brawner said he wished...",Executed by lethal injection in Mississippi on...,NOT FOUND,Parricide,WHITE,4
2,4 days after (surrenders),"October 17, 1964","November 1, 1990",None,"Sentenced to death on September 14, 1991. Exec...",NOT FOUND,Attempt to steal money kept in his mother’s sa...,BLACK,4
3,"November 21, 1995",1950,"February 22, 1974 / November 21, 1995",NOT FOUND,Sentenced to life imprisonment in 1975. Parole...,NOT FOUND,Rape,NOT FOUND,2
4,Two days after,"January 3, 1977","October 14, 1995",NOT FOUND,Executed by lethal injection at the Mississipp...,NOT FOUND,Rape,NOT FOUND,1
5,Next day,1973,"December 13, 1995",NOT FOUND,Executed by lethal injection at the Mississipp...,NOT FOUND,Robbery spree,NOT FOUND,2
6,Same day,"July 17, 1972","August 15, 1994",None.,Executed by lethal injection in Mississippi on...,NOT FOUND,Rape - Robbery,BLACK,1
7,Same day,"May 12, 1958","October 18, 1998",“I am sorry . . .Lord knows I am sorry. What I...,Executed by lethal injection in Mississippi on...,NOT FOUND,Revenge,NOT FOUND,4
8,NOT FOUND,"February 3, 1968","November 9, 1994",Burns recited the 23rd Psalm and said he was g...,Executed by lethal injection in Mississippi on...,NOT FOUND,Robbery,WHITE,1
9,NOT FOUND,"August 14, 1937","September 12, 1986","""I'm really deep down in my heart sorry it hap...",Executed by lethal injection in Mississippi on...,NOT FOUND,Rape,WHITE,1


In [182]:
df_all_mississippi['url'] = df_mississippi.link_to_profile 
df_all_mississippi = df_all_mississippi.merge(df_mississippi, right_on='link_to_profile', left_on = 'url')
df_all_mississippi['year'] = df_mississippi['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_mississippi.to_csv('mississippi-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Missouri

In [184]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/missouri.htm')

In [185]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Missouri"
    rows_df.append(row_df)
#print(rows_df)
df_missouri = pd.DataFrame(rows_df)
df_missouri

,State,county,date_execution,link_to_profile,method,name
0,Missouri,Boone,"September 10, 2014",http://murderpedia.org/male.R/r/ringo-earl.htm,Lethal Injection,Earl Ringo Jr.
1,Missouri,St. Charles,"August 6, 2014",http://murderpedia.org/male.W/w/worthington-mi...,Lethal Injection,Michael Shane Worthington
2,Missouri,Harrison,"July 16, 2014",http://murderpedia.org/male.M/m/middleton-john...,Lethal Injection,John C. Middleton
3,Missouri,St. Louis,"June 18, 2014",http://murderpedia.org/male.W/w/winfield-john.htm,Lethal Injection,John E. Winfield
4,Missouri,Washington,"April 23, 2014",http://murderpedia.org/male.R/r/rousan-william...,Lethal Injection,William L. Rousan
5,Missouri,St. Louis,"March 26, 2014",http://murderpedia.org/male.F/f/ferguson-jeffr...,Lethal Injection,Jeffrey R. Ferguson
6,Missouri,Jackson,"February 26, 2014",http://murderpedia.org/male.T/t/taylor-michael...,Lethal Injection,Michael Anthony Taylor
7,Missouri,St. Louis,"January 29, 2014",http://murderpedia.org/male.S/s/smulls-herbert...,Lethal Injection,Herbert L. Smulls
8,Missouri,Jackson,"December 11, 2013",http://murderpedia.org/male.N/n/nicklasson-all...,Lethal Injection,Allen L. Nicklasson
9,Missouri,St. Louis City,"November 20, 2013",http://murderpedia.org/male.F/f/franklin-josep...,Lethal Injection,Joseph Paul Franklin


In [186]:
df_all_missouri = df_missouri.apply(scrape_each_execution_arkansas, axis=1)
df_all_missouri

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,9 days after,"November 29, 1973","July 4, 1998",Ringo left behind a note with his final words ...,"Sentenced to death on July 26, 1999. Executed ...",NOT FOUND,Robbery,NOT FOUND,2
1,Next day (suicide attempt),"January 30, 1971","September 30, 1995",Worthington issued a final statement before hi...,"Sentenced to death on January 4, 1999. Execute...",NOT FOUND,Rape - Robbery,NOT FOUND,1
2,NOT FOUND,"November 2, 1959","June 11/23, 1995","""You are killing an innocent man.""","Sentenced to death on April 24, 1997. Executed...",NOT FOUND,User and dealer of methamphetamine - Revenge,NOT FOUND,3
3,Next day,"August 18, 1970","September 9, 1996",NOT FOUND,"Sentenced to death on September 18, 1998. Exec...",NOT FOUND,Jealousy,NOT FOUND,2
4,"September 20, 1994","July 17, 1956","September 21, 1993",NOT FOUND,"Sentenced to death on December 20, 1996. Execu...",NOT FOUND,Robbery,NOT FOUND,2
5,"February 22, 1989","January 24, 1955","February 10, 1989",NOT FOUND,"Sentenced to death on December 8, 1995. Execut...",NOT FOUND,Abduction - Rape,NOT FOUND,1
6,NOT FOUND,"January 30, 1967","January 24, 1995",None.,"Sentenced to death on May 14, 1991. Executed b...",NOT FOUND,Kidnapping - Rape,NOT FOUND,1
7,Same day,"November 28, 1957","July 27, 1991",None.,"Sentenced to death on December 21, 1992. Execu...",NOT FOUND,Robbery,NOT FOUND,1
8,NOT FOUND,"July 25, 1972","August 23/25, 1994",NOT FOUND,"Sentenced to death in Missouri on June 28, 199...",NOT FOUND,Robberies,NOT FOUND,3
9,"September 25, 1980","April 13, 1950",1977 - 1980,NOT FOUND,"Sentenced to death in Missouri on February 27,...",NOT FOUND,Former Klansman and neo-Nazi - Racially-motiva...,NOT FOUND,15 +


In [187]:
df_all_missouri['url'] = df_missouri.link_to_profile 
df_all_missouri = df_all_missouri.merge(df_missouri, right_on='link_to_profile', left_on = 'url')
df_all_missouri['year'] = df_missouri['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_missouri.to_csv('missouri-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Montana

In [188]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/montana.htm')

In [189]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Montana"
    rows_df.append(row_df)
#print(rows_df)
df_montana = pd.DataFrame(rows_df)
df_montana

,State,county,date_execution,link_to_profile,method,name
0,Montana,Yellowstone,"August 11, 2006",http://murderpedia.org/male.D/d1/dawson-david-...,Lethal Injection,David Thomas Dawson
1,Montana,Powell,"February 24, 1998",http://murderpedia.org/male.L/l1/langford-terr...,Lethal Injection,Terry Allen Langford
2,Montana,Pondera,"May 10, 1995",http://murderpedia.org/male.M/m1/mckenzie-dunc...,Lethal Injection,Duncan Peder McKenzie


In [190]:
df_all_montana = df_montana.apply(scrape_each_execution_arkansas, axis=1)
df_all_montana

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,2 days after,"October 20, 1957","April 19, 1986",Declined.,Executed by lethal injection in Montana on Aug...,NOT FOUND,Robbery,NOT FOUND,3
1,"August 12, 1988",1967,1988 / 1991,NOT FOUND,Executed by lethal injection in Montana on Feb...,NOT FOUND,Robbery,NOT FOUND,3
2,Next day,"October 5, 1951","January 21, 1974",NOT FOUND,Executed by lethal injection in Montana on May...,NOT FOUND,Rape - Torture,NOT FOUND,1


In [192]:
df_all_montana['url'] = df_montana.link_to_profile 
df_all_montana = df_all_montana.merge(df_montana, right_on='link_to_profile', left_on = 'url')
df_all_montana['year'] = df_montana['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_montana.to_csv('montana-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Nebraska

In [193]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/nebraska.htm')

In [194]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Nebraska"
    rows_df.append(row_df)
#print(rows_df)
df_nebraska = pd.DataFrame(rows_df)
df_nebraska

,State,county,date_execution,link_to_profile,method,name
0,Nebraska,Lancaster,"December 2, 1997",http://murderpedia.org/male.W/w1/williams-robe...,Electrocution,Robert E. Williams
1,Nebraska,Sarpy,"July 17, 1996",http://murderpedia.org/male.J/j1/joubert-john.htm,Electrocution,John Joseph Joubert IV
2,Nebraska,Douglas,"September 2, 1994",http://murderpedia.org/male.O/o1/otey-harold-l...,Electrocution,Harold Lamont Otey


In [195]:
df_all_nebraska = df_nebraska.apply(scrape_each_execution_arkansas, axis=1)
df_all_nebraska

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,"August 18, 1977",1936,"August 11/12, 1977",NOT FOUND,Executed by electrocution in Nebraska on Decem...,NOT FOUND,Rape,NOT FOUND,3
1,"January 11, 1984","July 2, 1963",1982 - 1983,NOT FOUND,Executed by electrocution in Nebraska on July ...,NOT FOUND,Kidnapping - Bites - Sadism,NOT FOUND,3 +
2,NOT FOUND,"August 1, 1951","June 11, 1977",NOT FOUND,Executed by electrocution in Nebraska on Septe...,NOT FOUND,Rape - Robbery,NOT FOUND,1


In [196]:
df_all_nebraska['url'] = df_nebraska.link_to_profile 
df_all_nebraska = df_all_nebraska.merge(df_nebraska, right_on='link_to_profile', left_on = 'url')
df_all_nebraska['year'] = df_nebraska['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_nebraska.to_csv('nebraska-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Nevada

In [197]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/nevada.htm')

In [198]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Nevada"
    rows_df.append(row_df)
#print(rows_df)
df_nevada = pd.DataFrame(rows_df)
df_nevada

,State,county,date_execution,link_to_profile,method,name
0,Nevada,Washoe,"April 26, 2006",http://murderpedia.org/male.M/m1/mack-daryl.htm,Lethal Injection,Darryll Linnie Mack
1,Nevada,Washoe,"August 12, 2004",http://murderpedia.org/male.D/d1/dennis-terry-...,Lethal Injection,Terry Jess Dennis
2,Nevada,Clark,"March 26, 2004",http://murderpedia.org/male.C/c1/colwell-lawre...,Lethal Injection,Lawrence Colwell Jr.
3,Nevada,Clark,"April 21, 2001",http://murderpedia.org/male.B/b1/bridges-sebas...,Lethal Injection,Sebastian Stephanus Bridges
4,Nevada,Washoe,"April 5, 1999",http://murderpedia.org/male.C/c1/calambro-alva...,Lethal Injection,Alvaro Calambro
5,Nevada,Clark,"October 5, 1998",http://murderpedia.org/male.A/a1/abeyta-roderi...,Lethal Injection,Roderick Abeyta
6,Nevada,Clark,"March 30, 1996",http://murderpedia.org/male.M/m1/moran-richard...,Lethal Injection,Richard Allen Moran
7,Nevada,Clark,"June 3, 1990",http://murderpedia.org/male.B/b1/baal-thomas.htm,Lethal Injection,Thomas E. Baal
8,Nevada,Clark,"June 23, 1989",http://murderpedia.org/male.F/f1/flanagan-sean...,Lethal Injection,Sean Patrick Flanagan
9,Nevada,Washoe,"June 19, 1989",http://murderpedia.org/male.T/t1/thompson-will...,Lethal Injection,William Paul Thompson


In [199]:
df_all_nevada = df_nevada.apply(scrape_each_execution_arkansas, axis=1)
df_all_nevada

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,NOT FOUND,"October 28, 1958",1988 / 1994,"""Allah is great, Allah is great"".",Executed by lethal injection in Nevada on Apri...,NOT FOUND,Rape,NOT FOUND,2
1,4 days after (surrenders),"October 14, 1946","March 5, 1999",None.,Executed by lethal injection in Nevada on Augu...,NOT FOUND,Alcohol,NOT FOUND,1
2,NOT FOUND,"March 1, 1969","March 10, 1994",None.,Executed by lethal injection in Nevada on Marc...,NOT FOUND,Robbery,NOT FOUND,1
3,Same day,"September 12, 1963","October 26, 1997",NOT FOUND,Executed by lethal injection in Nevada on Apri...,NOT FOUND,Revenge,NOT FOUND,1
4,NOT FOUND,1973,"January 3, 1994",NOT FOUND,Executed by lethal injection in Nevada on Apri...,NOT FOUND,Robbery - Borderline mentally retarded,NOT FOUND,2
5,NOT FOUND,1954,October 1989,NOT FOUND,Executed by lethal injection in Nevada on Octo...,NOT FOUND,Drugs,NOT FOUND,1
6,"August 13, 1984 (suicide attempt)",1954,"August 2/11, 1984",NOT FOUND,Executed by lethal injection in Nevada on Marc...,NOT FOUND,Robbery - State of cocaine-induced paranoia,NOT FOUND,3
7,2 days after,1965,"February 26, 1988",NOT FOUND,Executed by lethal injection in Nevada on June...,NOT FOUND,Robbery,NOT FOUND,1
8,NOT FOUND,1961,1987,NOT FOUND,Executed by lethal injection in Nevada on June...,NOT FOUND,"Dismemberment - ""Service a society by killing ...",NOT FOUND,2
9,NOT FOUND,1937,1983 - 1984,NOT FOUND,Executed by lethal injection in Nevada on June...,NOT FOUND,Murder for hire - Personal conflict,NOT FOUND,3 - 6


In [200]:
df_all_nevada['url'] = df_nevada.link_to_profile 
df_all_nevada = df_all_nevada.merge(df_nevada, right_on='link_to_profile', left_on = 'url')
df_all_nevada['year'] = df_nevada['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_nevada.to_csv('nevada-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# New Mexico

In [201]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/new_mexico.htm')

In [202]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "New Mexico"
    rows_df.append(row_df)
#print(rows_df)
df_newmexico = pd.DataFrame(rows_df)
df_newmexico

,State,county,date_execution,link_to_profile,method,name
0,New Mexico,Chaves,"November 6, 2001",http://murderpedia.org/male.C/c1/clark-terry.htm,Lethal Injection,Terry D. Clark


In [203]:
df_all_newmexico = df_newmexico.apply(scrape_each_execution_arkansas, axis=1)
df_all_newmexico

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,July 1986,"May 17, 1956","July 17, 1986",NOT FOUND,"Executed by lethal injection in November 6, 2001",NOT FOUND,Rape,NOT FOUND,1


In [205]:
df_all_newmexico['url'] = df_newmexico.link_to_profile 
df_all_newmexico = df_all_newmexico.merge(df_newmexico, right_on='link_to_profile', left_on = 'url')
df_all_newmexico['year'] = df_newmexico['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_newmexico.to_csv('newmexico.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# North Carolina

In [206]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/north_carolina.htm')

In [207]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "North Carolina"
    rows_df.append(row_df)
#print(rows_df)
df_northcarolina = pd.DataFrame(rows_df)
df_northcarolina

,State,county,date_execution,link_to_profile,method,name
0,North Carolina,Forsyth,"August 18, 2006",http://murderpedia.org/male.F/f1/flippen-samue...,Lethal Injection,Samuel Russell Flippen
1,North Carolina,Martin,"April 21, 2006",http://murderpedia.org/male.B/b1/brown-willie-...,Lethal Injection,Willie Brown Jr.
2,North Carolina,Davidson,"March 17, 2006",http://murderpedia.org/male.M/m1/moody-patrick...,Lethal Injection,Patrick Lane Moody
3,North Carolina,Rockingham,"January 20, 2006",http://murderpedia.org/male.S/s1/simpson-perri...,Lethal Injection,Perrie Dyon Simpson
4,North Carolina,Rockingham,"December 2, 2005",http://murderpedia.org/male.B/b1/boyd-kenneth-...,Lethal Injection,Kenneth Lee Boyd
5,North Carolina,Mecklenburg,"November 18, 2005",http://murderpedia.org/male.S/s1/syriani-elias...,Lethal Injection,Ellias Hanna Syriani
6,North Carolina,Surry,"November 11, 2005",http://murderpedia.org/male.V/v1/van-mchone-st...,Lethal Injection,Steven Van McHone
7,North Carolina,Cumberland,"May 6, 2005",http://murderpedia.org/male.R/r1/richmond-earl...,Lethal Injection,Earl J. Richmond Jr.
8,North Carolina,Chebeland,"March 11, 2005",http://murderpedia.org/male.P/p1/powell-willia...,Lethal Injection,William Dillard Powell
9,North Carolina,Surry,"November 12, 2004",http://murderpedia.org/male.C/c1/chandler-fran...,Lethal Injection,Frank Ray Chandler


In [208]:
df_all_northcarolina = df_northcarolina.apply(scrape_each_execution_arkansas, axis=1)
df_all_northcarolina

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,NOT FOUND,"September 30, 1969","February 12, 1994",Flippen did not make a final statement. He loc...,Executed by lethal injection in North Carolina...,NOT FOUND,Parricide,NOT FOUND,1
1,Same day,"November 24, 1944","March 6, 1983","""I love you.""",Executed by lethal injection in North Carolina...,NOT FOUND,Robbery,NOT FOUND,1
2,Next day,"July 4, 1966","September 16, 1994","""I'll see you in heaven. I'll see you up there.""",Executed by lethal injection in North Carolina...,NOT FOUND,Attempt to collect insurance money,NOT FOUND,1
3,"September 21, 1984","October 30, 1962","August 27, 1984","""I want to say I am sorry for what I did. I'm ...",Executed by lethal injection in North Carolina...,NOT FOUND,Robbery,NOT FOUND,1
4,Same day,"January 19, 1948","March 4, 1988","""I was just going to ask Kathy, my daughter-in...",Executed by lethal injection in North Carolina...,NOT FOUND,Parricide,NOT FOUND,2
5,Same day,"January 7, 1938","July 28, 1990","""I love you,"" he said to the Egglestons. ""I lo...",Executed by lethal injection in North Carolina...,NOT FOUND,Parricide,NOT FOUND,1
6,Same day,"March 23, 1970","June 3, 1990","He issued no last statement, but appeared to s...",Executed by lethal injection in North Carolina...,NOT FOUND,Parricide,NOT FOUND,2
7,"April 3, 1992","November 6, 1961",April/November 1991,"""At this time, I'd like to extend my deepest a...",Executed by lethal injection in North carolina...,NOT FOUND,Rape,NOT FOUND,4
8,"November 16, 1991","September 25, 1946","October 31, 1991",None.,Executed by lethal injection in North Carolina...,NOT FOUND,Robbery - Drugs,NOT FOUND,1
9,"January 5, 1993","October 7, 1972","December 11, 1992",None.,Executed by lethal injection in North Carolina...,NOT FOUND,Robbery,NOT FOUND,1


In [209]:
df_all_northcarolina['url'] = df_northcarolina.link_to_profile 
df_all_northcarolina = df_all_northcarolina.merge(df_northcarolina, right_on='link_to_profile', left_on = 'url')
df_all_northcarolina['year'] = df_northcarolina['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_northcarolina.to_csv('northcarolina.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Ohio

In [210]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/ohio.htm')

In [211]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Ohio"
    rows_df.append(row_df)
#print(rows_df)
df_ohio = pd.DataFrame(rows_df)
df_ohio

,State,county,date_execution,link_to_profile,method,name
0,Ohio,Preble,"January 16, 2014",http://murderpedia.org/male.M/m1/mcguire-denni...,Lethal Injection,Dennis B. McGuire
1,Ohio,Cuyahoga,"September 25, 2013",http://murderpedia.org/male.M/m1/mitts-harry.htm,Lethal Injection,Harry Mitts Jr.
2,Ohio,Richland,"May 5, 2013",http://murderpedia.org/male.S/s1/smith-steven.htm,Lethal Injection,Steven T. Smith
3,Ohio,Lake,"March 6, 2013",http://murderpedia.org/male.T/t1/treesh-freder...,Lethal Injection,Frederick Treesh
4,Ohio,Summit,"November 13, 2012",http://murderpedia.org/male.H/h1/hartman-brett...,Lethal Injection,Brett Xavier Hartman
5,Ohio,Belmont,"September 20, 2012",http://murderpedia.org/male.P/p1/palmer-donald...,Lethal Injection,Donald L. Palmer
6,Ohio,Portage,"April 18, 2012",http://murderpedia.org/male.W/w1/wiles-mark.htm,Lethal Injection,Mark Wayne Wiles
7,Ohio,Cuyahoga,"November 15, 2011",http://murderpedia.org/male.B/b1/brooks-regina...,Lethal Injection,Reginald Brooks
8,Ohio,Hamilton,"May 17, 2011",http://murderpedia.org/male.B/b1/bedford-danie...,Lethal Injection,Daniel Lee Bedford
9,Ohio,Hamilton,"April 12, 2011",http://murderpedia.org/male.C/c1/carter-claren...,Lethal Injection,Clarence Carter


In [212]:
df_all_ohio = df_ohio.apply(scrape_each_execution_arkansas, axis=1)
df_all_ohio

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,"December 22, 1993","February 10, 1960","February 11, 1989",NOT FOUND,Executed by lethal injection in Ohio on Januar...,NOT FOUND,Kidnapping - Rape,NOT FOUND,1
1,Same day (wounded by police),"June 18, 1942","August 14, 1994",NOT FOUND,"Sentenced to death on November 21, 1994. Execu...",NOT FOUND,Shooting rampage - Hate crime,NOT FOUND,2
2,Next day,"February 9, 1967","September 29, 1998",None.,"Sentenced to death on March 25, 1999. Executed...",NOT FOUND,Rape,White,1
3,"August 27, 1994","April 12, 1964","August 25-27, 1994",“This is where drugs lead you.” Treesh apologi...,"Sentenced to death on March 2, 1995. Executed ...",NOT FOUND,Crime spree - Robberies,White,1 - 2
4,Next day,"June 29, 1974","September 9, 1997","“I’m good, let’s roll"".","Sentenced to death on May 22, 1998. Executed b...",NOT FOUND,Kidnapping - Cut off her hands (they were neve...,White,1
5,"May 23, 1989","February 11, 1965","May 8, 1989","""I want you to know I've carried you in my hea...","Sentenced to death on October 26, 1999. Execut...",NOT FOUND,Argument - Robbery,NOT FOUND,2
6,5 days after (surrenders),"March 11, 1963","August 7, 1985","""Since this needs to be happening, truly I pra...",Executed by lethal injection in Ohio on April ...,NOT FOUND,Burglary,White,1
7,Same,"March 20, 1945","March 6, 1982","None, but took the opportunity to give the mid...",Executed by lethal injection in Ohio on Novemb...,NOT FOUND,Parricide- After his wife filed for divorce,Black,3
8,Same day,"September 16, 1947","April 24, 1984","""I love you, Shell. Love you all. God bless you.""",Executed by lethal injection in Ohio on May 17...,NOT FOUND,Angered because his former girlfriend ended th...,White,2
9,NOT FOUND,"March 9, 1962","August 13, 1988 / January 11, 1989","""I'd like to say I'm sorry for what I did, esp...",Executed by lethal injection in Ohio on April ...,NOT FOUND,Murder for hire - Argument,Black,2


In [213]:
df_all_ohio['url'] = df_ohio.link_to_profile 
df_all_ohio = df_all_ohio.merge(df_ohio, right_on='link_to_profile', left_on = 'url')
df_all_ohio['year'] = df_ohio['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_ohio.to_csv('ohio-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Oklahoma

In [214]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/oklahoma.htm')

In [215]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Oklahoma"
    rows_df.append(row_df)
#print(rows_df)
df_oklahoma = pd.DataFrame(rows_df)
df_oklahoma

,State,county,date_execution,link_to_profile,method,name
0,Oklahoma,Kay,"April 30, 2014",http://murderpedia.org/male.L/l1/lockett-clayt...,Lethal Injection,Clayton Derrell Lockett
1,Oklahoma,Oklahoma,"January 23, 2014",http://murderpedia.org/male.H/h1/hogan-kenneth...,Lethal Injection,Kenneth Eugene Hogan
2,Oklahoma,Tulsa,"January 9, 2014",http://murderpedia.org/male.W/w1/wilson-michae...,Lethal Injection,Michael Lee Wilson
3,Oklahoma,Stephens,"December 17, 2013",http://murderpedia.org/male.B/b1/black-johnny.htm,Lethal Injection,Johnny Dale Black
4,Oklahoma,Oklahoma,"December 10, 2013",http://murderpedia.org/male.L/l1/lott-ronald.htm,Lethal Injection,Ronald Clinton Lott
5,Oklahoma,Tulsa,"September 10, 2013",http://murderpedia.org/male.B/b1/banks-anthony...,Lethal Injection,Anthony Rozelle Banks
6,Oklahoma,Kay,"June 25, 2013",http://murderpedia.org/male.D/d1/davis-brian-d...,Lethal Injection,Brian Darrell Davis
7,Oklahoma,LeFlore,"June 18, 2013",http://murderpedia.org/male.D/d1/derosa-james.htm,Lethal Injection,James Lewis DeRosa
8,Oklahoma,Mayes,"March 12, 2013",http://murderpedia.org/male.T/t/thacker-steven...,Lethal Injection,Steven Ray Thacker
9,Oklahoma,Oklahoma,"December 4, 2012",http://murderpedia.org/male.O/o1/ochoa-george.htm,Lethal Injection,George Ochoa


In [218]:
df_all_oklahoma = df_oklahoma.apply(scrape_each_execution_arkansas, axis=1)
df_all_oklahoma

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,Next day,"November 22, 1975","June 3, 1999",None.,"Sentenced to death on October 5, 2000. Execute...",NOT FOUND,Robbery - Kidnapping - Rape,Black,1
1,"February 3, 1988","May 22, 1961","January 28, 1988",NOT FOUND,"Sentenced to death on November 9, 1988. Convic...",NOT FOUND,Argument,NOT FOUND,1
2,Same day,"February 18, 1975","February 26, 1995",NOT FOUND,Executed by lethal injection in Oklahoma on Ja...,NOT FOUND,Robbery,NOT FOUND,1
3,NOT FOUND,"September 27, 1965","December 13, 1983 / January 4, 1998",NOT FOUND,"Convicted of manslaughter on December 28, 1984...",NOT FOUND,Case of mistaken identity,NOT FOUND,2
4,"March 19, 1997","September 22, 1960","September 2, 1986 / January 11, 1987",NOT FOUND,Sentenced to death in January 2002. Executed b...,NOT FOUND,Serial rapist,NOT FOUND,2
5,NOT FOUND,"July 5, 1952","April 11, 1978 / June 6, 1979",NOT FOUND,"Sentenced to death on November 22, 1999. Execu...",NOT FOUND,Armed robbery - Rape,NOT FOUND,2
6,Same day (injured in a single-car accident),"May 10, 1974","November 4, 2001",“First I would like to say I would like to giv...,"Sentenced to death on March 17, 2003. Executed...",NOT FOUND,Rape,Black,1
7,3 days after,"March 17, 1977","October 2, 2000",NOT FOUND,Executed by lethal injection in Oklahoma on Ju...,NOT FOUND,Robbery,NOT FOUND,2
8,"January 2, 2000","November 20, 1970","December 23, 1999-January 2, 2000",“I’d like to apologize sincerely to the famili...,"Sentenced to death in Tennessee on February 8,...",NOT FOUND,Rape - Robberies,White,3
9,Same day,"August 6, 1964","July 12, 1993","“I’m innocent.""","Sentenced to death on March 21, 1996. Executed...",NOT FOUND,Burglary,Hispanic,2


In [219]:
df_all_oklahoma['url'] = df_oklahoma.link_to_profile 
df_all_oklahoma = df_all_oklahoma.merge(df_oklahoma, right_on='link_to_profile', left_on = 'url')
df_all_oklahoma['year'] = df_oklahoma['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_oklahoma.to_csv('oklahoma-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Oregon

In [220]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/oregon.htm')

In [221]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Oregon"
    rows_df.append(row_df)
#print(rows_df)
df_oregon = pd.DataFrame(rows_df)
df_oregon

,State,county,date_execution,link_to_profile,method,name
0,Oregon,Wasco,"May 16, 1997",http://murderpedia.org/male.W/w1/wright-dougla...,Lethal Injection,Douglas Franklin Wright
1,Oregon,Marion,"September 6, 1996",http://murderpedia.org/male.M/m1/moore-harry-c...,Lethal Injection,Harry Charles Moore


In [222]:
df_all_oregon = df_oregon.apply(scrape_each_execution_arkansas, axis=1)
df_all_oregon

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,NOT FOUND,"March 25, 1940",1969 / 1984 / 1991,NOT FOUND,Executed by lethal injection in Oregon on Sept...,NOT FOUND,NOT FOUND,NOT FOUND,7 +
1,NOT FOUND,"May 5, 1941",June 1992,NOT FOUND,Executed by lethal injection in Oregon on May ...,NOT FOUND,Revenge,NOT FOUND,2


In [223]:
df_all_oregon['url'] = df_oregon.link_to_profile 
df_all_oregon = df_all_oregon.merge(df_oregon, right_on='link_to_profile', left_on = 'url')
df_all_oregon['year'] = df_oregon['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_oregon.to_csv('oregon-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Pennsylvania

In [224]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/pennsylvania.htm')

In [225]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Pennsylvania"
    rows_df.append(row_df)
#print(rows_df)
df_pennsylvania = pd.DataFrame(rows_df)
df_pennsylvania

,State,county,date_execution,link_to_profile,method,name
0,Pennsylvania,Philadelphia,"July 6, 1999",http://murderpedia.org/male.H/h1/heidnik-gary.htm,Lethal Injection,Gary Michael Heidnik
1,Pennsylvania,Philadelphia,"August 15, 1995",http://murderpedia.org/male.M/m1/moser-leon-je...,Lethal Injection,Leon Jerome Moser
2,Pennsylvania,Dauphin,"May 2, 1995",http://murderpedia.org/male.Z/z1/zettlemoyer-k...,Lethal Injection,Keith W. Zettlemoyer


In [226]:
df_all_pennsylvania = df_pennsylvania.apply(scrape_each_execution_arkansas, axis=1)
df_all_pennsylvania

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,"March 24, 1987","November 22, 1943",1986 - 1987,NOT FOUND,Executed by lethal injection in Pennsylvania o...,NOT FOUND,Cannibalism - Kidnapping - Torture - Serial ra...,NOT FOUND,2
1,NOT FOUND,1943,1985,NOT FOUND,Executed by lethal injection in Pennsylvania o...,NOT FOUND,Parricide,NOT FOUND,3
2,Same day,1956,"October 13, 1980",NOT FOUND,Executed by lethal injection in Pennsylvania o...,NOT FOUND,To avoid testifying against him,NOT FOUND,1


In [227]:
df_all_pennsylvania['url'] = df_pennsylvania.link_to_profile 
df_all_pennsylvania = df_all_pennsylvania.merge(df_pennsylvania, right_on='link_to_profile', left_on = 'url')
df_all_pennsylvania['year'] = df_pennsylvania['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_pennsylvania.to_csv('pennsylvania-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# South Carolina

In [228]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/south_carolina.htm')

In [229]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "South Carolina"
    rows_df.append(row_df)
#print(rows_df)
df_southcarolina = pd.DataFrame(rows_df)
df_southcarolina

,State,county,date_execution,link_to_profile,method,name
0,South Carolina,Greenville,"May 6, 2011",http://murderpedia.org/male.M/m1/motts-jeffrey...,Lethal Injection,Jeffrey Motts
1,South Carolina,Orangeburg,"May 8, 2009",http://murderpedia.org/male.I/i1/ivey-thomas-t...,Lethal Injection,Thomas Treshawn Ivey
2,South Carolina,Edgefield,"February 20, 2009",http://murderpedia.org/male.W/w1/williams-luke...,Lethal Injection,Luke A. Williams III
3,South Carolina,Dorchester,"December 5, 2008",http://murderpedia.org/male.G/g1/gardner-josep...,Lethal Injection,Joseph Martin Luther Gardner
4,South Carolina,Charleston,"June 20, 2008",http://murderpedia.org/male.R/r1/reed-james-ea...,Electrocution,James Earl Reed
5,South Carolina,Aiken,"June 6, 2008",http://murderpedia.org/male.H/h1/hill-david-ma...,Lethal Injection,David Mark Hill
6,South Carolina,Dorchester,"June 22, 2007",http://murderpedia.org/male.S/s1/shuler-calvin...,Lethal Injection,Calvin Alphonso Shuler
7,South Carolina,Aiken,"July 14, 2006",http://murderpedia.org/male.D/d1/downs-william...,Lethal Injection,William E. Downs Jr.
8,South Carolina,Greenville,"December 2, 2005",http://murderpedia.org/male.H/h1/humphries-sha...,Lethal Injection,Shawn Paul Humphries
9,South Carolina,Aiken,"November 4, 2005",http://murderpedia.org/male.W/w1/wise-hastings...,Lethal Injection,Hastings Arthur Wise


In [230]:
df_all_southcarolina = df_southcarolina.apply(scrape_each_execution_arkansas, axis=1)
df_all_southcarolina

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,NOT FOUND,"May 5, 1975","April 3, 1995 / December 5, 2005",His attorney read a last statement from Motts ...,Executed by lethal injection in South Carolina...,NOT FOUND,Robbery - Argument,White,3
1,NOT FOUND,"May 27, 1974","??? / January 13/15, 1993",None.,Executed by lethal injection in South Carolina...,NOT FOUND,Fugitive - Kidnapping - Robbery,Black,3
2,NOT FOUND,"August 14, 1952","June 19, 1991",None.,Executed by lethal injection in South Carolina...,NOT FOUND,Parricide - To collect insurance money,White,2
3,"October 20, 1994","January 15, 1970","December 30, 1992","Written statement only: ""I would like to apolo...",Executed by lethal injection in South Carolina...,NOT FOUND,Rape - Torture,Black,1
4,NOT FOUND,"November 26, 1958","May 15, 1994",None.,Executed by electrocution in South Carolina on...,NOT FOUND,Revenge,Black,2
5,Next day (suicide attempt),"May 24, 1960","September 16, 1996","""I sincerely apologize to the families of Josi...",Executed by lethal injection in South Carolina...,NOT FOUND,Revenge,White,3
6,NOT FOUND,"December 3, 1966","December 3, 1997",None.,Executed by lethal injection in South Carolina...,NOT FOUND,Armored car robbery,Black,1
7,NOT FOUND,"July 12, 1967",1991 / 1999,None.,Executed by lethal injection in South Carolina...,NOT FOUND,Rape - Child Molester,White,2
8,Same day,"October 19, 1971","January 1, 1994","Humphries looked toward the victim's sisters, ...",Executed by lethal injection in South Carolina...,NOT FOUND,Robbery,White,1
9,Same day (suicide attempt),"February 16, 1954","September 15, 1997",None.,Executed by lethal injection in South Carolina...,NOT FOUND,Revenge,Black,4


In [231]:
df_all_southcarolina['url'] = df_southcarolina.link_to_profile 
df_all_southcarolina = df_all_southcarolina.merge(df_southcarolina, right_on='link_to_profile', left_on = 'url')
df_all_southcarolina['year'] = df_southcarolina['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_southcarolina.to_csv('southcarolina-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# South Dakota

In [232]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/south_dakota.htm')

In [233]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "South Dakota"
    rows_df.append(row_df)
#print(rows_df)
df_southdakota = pd.DataFrame(rows_df)
df_southdakota

,State,county,date_execution,link_to_profile,method,name
0,South Dakota,Lawrence,"July 11, 2007",http://murderpedia.org/male.P/p1/page-elijah.htm,Lethal Injection,Elijah Page
1,South Dakota,Minnehaha,"October 15, 2012",http://murderpedia.org/male.R/r1/robert-eric.htm,Lethal Injection,Eric Donald Robert
2,South Dakota,Lincoln,"October 30, 2012",http://murderpedia.org/male.M/m1/moeller-donal...,Lethal Injection,Donald Eugene Moeller


In [234]:
df_all_southdakota = df_southdakota.apply(scrape_each_execution_arkansas, axis=1)
df_all_southdakota

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,NOT FOUND,1981,"March 12, 2000","Asked if he had any last words, Page replied, ...",Executed by lethal injection in South Dakota o...,NOT FOUND,Robbery - Torture,NOT FOUND,1
1,Same day,"May 31, 1962","April 12, 2011","“In the name of justice and liberty and mercy,...","Sentenced to death on October 20, 2011. Execut...",NOT FOUND,Failed escape attempt,NOT FOUND,1
2,"March 6, 1991","August 5, 1952","May 8, 1990",When asked whether he had any last words befor...,Sentenced to death in 1992 and 1997. Executed ...,NOT FOUND,Kidnapping - Rape,NOT FOUND,1


In [235]:
df_all_southdakota['url'] = df_southdakota.link_to_profile 
df_all_southdakota = df_all_southdakota.merge(df_southdakota, right_on='link_to_profile', left_on = 'url')
df_all_southdakota['year'] = df_southdakota['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_southdakota.to_csv('southdakota-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Tennessee

In [236]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/tennessee.htm')

In [237]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Tennessee"
    rows_df.append(row_df)
#print(rows_df)
df_tennessee = pd.DataFrame(rows_df)
df_tennessee

,State,county,date_execution,link_to_profile,method,name
0,Tennessee,Davidson,"December 2, 2009",http://murderpedia.org/male.J/j1/johnson-cecil...,Lethal Injection,Cecil J. Johnson Jr.
1,Tennessee,Jackson,"February 4, 2009",http://murderpedia.org/male.H/h1/henley-steve.htm,Lethal Injection,Steve Henley
2,Tennessee,Bedford,"May 12, 2007",http://murderpedia.org/male.H/h1/holton-daryl.htm,Electrocution,Daryl Keith Holton
3,Tennessee,Shelby,"May 9, 2007",http://murderpedia.org/male.W/w1/workman-phili...,Lethal Injection,Philip Ray Workman
4,Tennessee,Shelby,"June 28, 2006",http://murderpedia.org/male.A/a1/alley-sedley.htm,Lethal Injection,Sedley Alley
5,Tennessee,Weakley,"April 19, 2000",http://murderpedia.org/male.C/c1/coe-robert-gl...,Lethal Injection,Robert Glen Coe


In [238]:
df_all_tennessee = df_tennessee.apply(scrape_each_execution_arkansas, axis=1)
df_all_tennessee

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,"July 6, 1980","August 29, 1956","July 5, 1980 / 1985","Johnson mouthed “I love you,’’ repeatedly to h...",Executed by lethal injection in Tennessee on D...,NOT FOUND,Robbery,NOT FOUND,4
1,NOT FOUND,"November 25, 1953","July 24, 1985","“As I have said ever since this happened, I di...",Executed by lethal injection in Tennessee on F...,NOT FOUND,Robbery - Arson,NOT FOUND,2
2,Same day (surrenders),1962,"November 30, 1997","Asked if he had any last words, Holton replied...",Executed by electrocution in Tennessee on Sept...,NOT FOUND,Parricide - Gulf War Veteran,NOT FOUND,4
3,NOT FOUND,1953,"August 5, 1981",“I've prayed to the Lord Jesus Christ not to l...,Executed by lethal injection in Tennessee on M...,NOT FOUND,Robbery - To avoid arrest,NOT FOUND,1
4,Same day,"August 16, 1955","July 11, 1985","""Yes, to my children. April, David, can you he...",Executed by lethal injection in Tennessee on J...,NOT FOUND,Rape,NOT FOUND,1
5,3 days after,"April 15, 1956","September 1, 1979",NOT FOUND,Executed by lethal injection in Tennessee on A...,NOT FOUND,Rape,NOT FOUND,1


In [239]:
df_all_tennessee['url'] = df_tennessee.link_to_profile 
df_all_tennessee = df_all_tennessee.merge(df_tennessee, right_on='link_to_profile', left_on = 'url')
df_all_tennessee['year'] = df_tennessee['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_tennessee.to_csv('tennessee-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Utah

In [240]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/utah.htm')

In [242]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Utah"
    rows_df.append(row_df)
#print(rows_df)
df_utah = pd.DataFrame(rows_df)
df_utah

,State,county,date_execution,link_to_profile,method,name
0,Utah,Salt Lake,"June 18, 2010",http://murderpedia.org/male.G/g1/gardner-ronni...,Firing Squad,Ronnie Lee Gardner
1,Utah,Iron,"October 15, 1999",http://murderpedia.org/male.P/p1/parsons-josep...,Lethal Injection,Joseph Mitchell Parson
2,Utah,Weber,"January 27, 1996",http://murderpedia.org/male.T/t1/taylor-john-a...,Lethal Injection,John Albert Taylor
3,Utah,Weber,"July 30, 1992",http://murderpedia.org/male.A/a1/andrews-willi...,Lethal Injection,Williams Andrews
4,Utah,Salt Lake,"June 10, 1988",http://murderpedia.org/male.B/b1/bishop-arthur...,Lethal Injection,Arthur Gary Bishop
5,Utah,Weber,"August 28, 1987",http://murderpedia.org/male.S/s1/selby-pierre-...,Lethal Injection,Pierre Dale Selby
6,Utah,Salt Lake,"January 17, 1977",http://murderpedia.org/male.G/g1/gilmore-gary-...,Firing Squad,Gary Mark Gilmore


In [244]:
df_all_utah = df_utah.apply(scrape_each_execution_arkansas, axis=1)
df_all_utah

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,NOT FOUND,"January 16, 1961","October 9, 1984 / April 2, 1985",None.,"Executed by shooting in Utah on June 18, 2010",NOT FOUND,Robbery - Escape attempt,NOT FOUND,2
1,NOT FOUND,1964,"August 31, 1987",NOT FOUND,Executed by lethal injection in Utah on Octobe...,NOT FOUND,Robbery - Hitchhiking,NOT FOUND,1
2,5 days after,June 1959,"June 23, 1988",NOT FOUND,"Executed by shooting in Utah on January 26, 1996",NOT FOUND,Rape,NOT FOUND,1
3,Next day,1954,"April 22, 1974",NOT FOUND,Executed by lethal injection in Utah on July 3...,NOT FOUND,Robbery - Rape,NOT FOUND,1
4,July 1983,1951,1979 - 1983,NOT FOUND,Executed by lethal injection in in Utah on Jun...,NOT FOUND,Child molester,NOT FOUND,5
5,Next day,1952,"April 22, 1974",NOT FOUND,Executed by lethal injection in Utah on August...,NOT FOUND,Robbery - Rape,NOT FOUND,3
6,"July 21, 1976","December 4, 1940","July 19/20, 1976",NOT FOUND,"Executed by shooting in Utah on January 17, 1977",NOT FOUND,Robberies,NOT FOUND,2


In [245]:
df_all_utah['url'] = df_utah.link_to_profile 
df_all_utah = df_all_utah.merge(df_utah, right_on='link_to_profile', left_on = 'url')
df_all_utah['year'] = df_utah['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_utah.to_csv('utah-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Washington

In [246]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/washington.htm')

In [247]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Washington"
    rows_df.append(row_df)
#print(rows_df)
df_washington = pd.DataFrame(rows_df)
df_washington

,State,county,date_execution,link_to_profile,method,name
0,Washington,King,"September 10, 2010",http://murderpedia.org/male.B/b1/brown-cal-cob...,Lethal Injection,Cal Coburn Brown
1,Washington,Snohomish,"August 28, 2001",http://murderpedia.org/male.E/e1/elledge-james...,Lethal Injection,James Homer Elledge
2,Washington,Benton,"October 13, 1998",http://murderpedia.org/male.S/s1/sagastegui-je...,Lethal Injection,Jeremy V. Sagastegui
3,Washington,Snohomish,"May 27, 1994",http://murderpedia.org/male.C/c1/campbell-char...,Hanging,Charles Rodman Campbell
4,Washington,Clark,"January 5, 1993",http://murderpedia.org/male.D/d1/dodd-westley-...,Hanging,Westley Allan Dodd


In [248]:
df_all_washington = df_washington.apply(scrape_each_execution_arkansas, axis=1)
df_all_washington

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,3 days after,"May 16, 1958","May 24, 1991","In a lengthy statement, Brown did not apologiz...",Executed by lethal injection in Washington on ...,NOT FOUND,Carjacking - Rape - Torture,White,1
1,"April 21, 1998",1943,1975 / 1998,NOT FOUND,Executed by lethal injection in Washington on ...,NOT FOUND,Argument - Rape - Revenge,NOT FOUND,2
2,Next day,"November 1, 1970","November 19, 1995",NOT FOUND,Executed by lethal injection in Washington on ...,NOT FOUND,Rape,NOT FOUND,3
3,NOT FOUND,"October 21, 1954","April 14, 1982",NOT FOUND,"Executed by hanging in Washington on May 27, 1994",NOT FOUND,Rape - Revenge,NOT FOUND,3
4,"November 13, 1989","July 3, 1961","September 4/October 29,1989",NOT FOUND,Executed by hanging in Washington on January 5...,NOT FOUND,Kidnapping - Rape - Child molester,NOT FOUND,3


In [249]:
df_all_washington['url'] = df_washington.link_to_profile 
df_all_washington = df_all_washington.merge(df_washington, right_on='link_to_profile', left_on = 'url')
df_all_washington['year'] = df_washington['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_washington.to_csv('washington-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# Wyoming

In [250]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/wyoming.htm')

In [251]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Wyoming"
    rows_df.append(row_df)
#print(rows_df)
df_wyoming = pd.DataFrame(rows_df)
df_wyoming

,State,county,date_execution,link_to_profile,method,name
0,Wyoming,Uinta,"January 22, 1992",http://murderpedia.org/male.H/h1/hopkinson-mar...,Lethal Injection,Mark A. Hopkinson


In [252]:
df_all_wyoming = df_wyoming.apply(scrape_each_execution_arkansas, axis=1)
df_all_wyoming

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,NOT FOUND,"October 8, 1949",1977 / 1979,NOT FOUND,Executed by lethal injection in Wyoming on Jan...,NOT FOUND,Revenge - Murder for hire,NOT FOUND,4


In [253]:
df_all_wyoming['url'] = df_wyoming.link_to_profile 
df_all_wyoming = df_all_wyoming.merge(df_wyoming, right_on='link_to_profile', left_on = 'url')
df_all_wyoming['year'] = df_wyoming['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_wyoming.to_csv('wyoming-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


# AND FINALLY VIRGINIA

In [254]:
driver = webdriver.Chrome()
driver.get('http://deathpenaltyusa.org/usa/state/virginia.htm')

In [255]:
table = driver.find_element_by_id('table13')
rows = table.find_elements_by_tag_name('tr')
rows_df = []
for row in rows[1:]:
    cells = row.find_elements_by_tag_name('td')
    row_df = {}
    #for cell in cells:
        #print(cell.text)
    row_df['name'] = cells[1].text
    row_df['link_to_profile'] = cells[1].find_element_by_tag_name('a').get_attribute('href')
    row_df['date_execution'] = cells[2].text
    row_df['method'] = cells[3].text
    row_df['county'] = cells[4].text
    row_df['State'] = "Virginia"
    rows_df.append(row_df)
#print(rows_df)
df_virginia = pd.DataFrame(rows_df)
df_virginia

,State,county,date_execution,link_to_profile,method,name
0,Virginia,Williamsburg,"January 16, 2013",http://murderpedia.org/male.G/g1/gleason-rober...,Electrocution,Robert Charles Gleason Jr.
1,Virginia,Williamsburg,"August 18, 2011",http://murderpedia.org/male.J/j1/jackson-jerry...,Lethal Injection,Jerry Terrell Jackson
2,Virginia,Pittsylvania,"September 23, 2010",http://deathpenaltyusa.org/usa/female/lewis-te...,Lethal Injection,Teresa Wilson Bean Lewis
3,Virginia,Henrico,"May 20, 2010",http://murderpedia.org/male.W/w1/walker-darick...,Lethal Injection,Darick Demorris Walker
4,Virginia,Prince William,"March 18, 2010",http://murderpedia.org/male.P/p1/powell-paul-w...,Electrocution,Paul Warner Powell
5,Virginia,Prince William,"November 17, 2009",http://murderpedia.org/male.E/e1/elliot-larry-...,Electrocution,Larry Bill Elliott
6,Virginia,Prince William,"November 10, 2009",http://murderpedia.org/male.M/m1/muhammad-john...,Lethal Injection,John Allen Muhammad
7,Virginia,Winchester,"February 19, 2009",http://murderpedia.org/male.B/b1/bell-edward-n...,Lethal Injection,Edward Nathaniel Bell
8,Virginia,Pittsylvania,"July 24, 2008",http://murderpedia.org/male.E/e1/emmett-christ...,Lethal Injection,Christopher Scott Emmett
9,Virginia,Warwick,"July 10, 2008",http://murderpedia.org/male.J/j1/jackson-kent-...,Lethal Injection,Kent Jermaine Jackson


In [256]:
df_all_virginia = df_virginia.apply(scrape_each_execution_arkansas, axis=1)
df_all_virginia

,date_arrest,date_of_birth,date_offense,final_words,method,mitigating_circumstances,offense,race,victim_num
0,NOT FOUND,"April 10, 1970",2007 / 2009 / 2010,"In Irish Gaelic, the phrase “Pog mo thoin,” tr...","Sentenced to death on September 6, 2011. Execu...",NOT FOUND,Killed two fellow inmates while serving a life...,NOT FOUND,3
1,NOT FOUND,"July 22, 1981","August 25, 2001",None,"Sentenced to death on April 3, 2003. Executed ...",NOT FOUND,Rape - Robbery,Black,1
2,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND,NOT FOUND
3,NOT FOUND,"July 8, 1972","November 22, 1996 / June 18, 1997","""Last words being: I don't think y'all done th...",Executed by lethal injection in Virginia on Ma...,NOT FOUND,Mentally retarded,NOT FOUND,2
4,Next day,"April 13, 1978","January 29, 1999",Declined.,Executed by electrocution in Virginia on March...,NOT FOUND,Rape - White supremacist,NOT FOUND,1
5,NOT FOUND,"November 13, 1949","January 2, 2001","None. Shortly after the execution, his attorne...",Executed by electrocution in Virginia on Novem...,NOT FOUND,Internet - Jealousy,White,2
6,"October 24, 2002","December 31, 1960",September-October 2002,None.,Executed by lethal injection in Virginia on No...,NOT FOUND,Sniper attacks,NOT FOUND,10 +
7,Same day,"September 12, 1964","October 29, 1999","""To the Timbrook family, you definitely have t...",Executed by lethal injection in Virginia on Fe...,NOT FOUND,To avoid arrest,Black,1
8,Same day,"August 18, 1971","April 27, 2001","""Tell my family and friends I love them, tell ...",Executed by lethal injection in Virginia on Ju...,NOT FOUND,Robbery - Drugs,White,1
9,"August 28, 2001","September 18, 1981","April 18, 2000","""You all can't kill me. I'm the king. Remember...",Executed by lethal injection in Virginia on Ju...,NOT FOUND,Rape - Robbery,Black,1


In [257]:
df_all_virginia['url'] = df_virginia.link_to_profile 
df_all_virginia = df_all_virginia.merge(df_virginia, right_on='link_to_profile', left_on = 'url')
df_all_virginia['year'] = df_virginia['date_execution'].str.extract(r"(\d\d\d\d)")
df_all_virginia.to_csv('virginia-2.csv', index=False)

/Users/mariepatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until
